# INSTRUCTIONS

**Use 'ctrl+]' to collapse all** if running in Google Colab

Steps to be followed

1. Mount your drive containing data.
2. Set 'prjct_dir' and 'data_dir' arguments in argparser. 
3. Run all cells before "Final results" section.
4. Run the exp(experiment) required in "Final results" section. 

Note: 
* Arguments except 'prjct_dir' and 'data_dir' will be set automatically according to selected exp in "Final results"

* *seq_len(no. of sequence of samples used for rnn/lstm)*
**  Offline: data is coming from both side. Therefore, No. of sequence of samples used for rnn/lstm = seq_len taken
** Online: data is coming from one side.
No. of sequence of samples used for rnn/lstm = (seq_len+1)/2, i.e. if seq_len = 7 then  no. of samples used for rnn/lstm = 4
* SNR(signal to noise ratio)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).



# ARGPARSE

In [3]:
! pip install cmocean
import pdb
import pandas as pd
import yaml
import h5py
import numpy as np
from numpy import sum, isrealobj, sqrt
from numpy.random import standard_normal
import scipy.linalg as la

import os
import os.path as osp
from os.path import dirname

import torch
import torch as T
from torch import nn
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import ExponentialLR
from torch.optim.lr_scheduler import MultiplicativeLR
from torch.optim.lr_scheduler import LambdaLR
import torch.nn.functional as F
from torch.utils.data.dataset import Dataset
from torch.autograd import Variable
from torch.nn.parameter import Parameter, UninitializedParameter

import matplotlib.pyplot as plt
from matplotlib import rc
from matplotlib import cm
import matplotlib as mpl
import cmocean
import time
import random
import argparse
from itertools import count
from itertools import chain
from skimage.transform import resize
from secrets import token_hex

In [4]:
"""
Change directory path 
prjct_dir: saving weights and results 
data_dir: data 
"""
parser = argparse.ArgumentParser('ARE')

prjct_dir = '../content/drive/My Drive/Colab Notebooks/are'
data_dir = '../content/drive/My Drive/Data/are_data'

parser.add_argument('--prjct_dir', type=str, default=prjct_dir)
parser.add_argument('--data_dir', type=str, default=data_dir)

parser.add_argument('--exp', type=str, choices=['mthd__snsr', 'snsr__seq_len', 'mthd__snr', 
                                                'mthd__tr_samp', 'mthd__bn',
                                                'rndm_trials', 'mthd__snsr_with_noise'], default='mthd__snsr')
parser.add_argument('--operation_mode', type=str, choices=['offline', 'online'], default='online')  
parser.add_argument('--RNN', type=str, choices=['lstm', 'rnn'], default='lstm')
parser.add_argument('--gpu', type=int, default=0)
args = parser.parse_args("")

args.device = device = torch.device('cuda:' + str(args.gpu) if torch.cuda.is_available() else 'cpu')

# Utils

In [5]:
def get_path(method, string, **kwargs):
    """ 
    determine path to directory based upon string
    e.g. W=1 in kwargs return path of dir. to save weights

    Returns: path to directory
    """

    dir1 = args.data_type
    dir2 = args.exp
    dir3 = osp.join(method, args.operation_mode, args.RNN) if method == 'are' else method 
    if args.exp == 'mthd__tr_samp':
        if hasattr(args, 'tr_samp'): add = 'tr_samp{}'.format(args.tr_samp)  
        else: raise Exception('number of training samples are required')
    if args.exp == 'mthd__snsr_loc':
        seed = kwargs.get('seed', 0)
        dir3 = osp.join(dir3, f'seed{seed}')
    if args.exp == 'SDnets':
        SDnet = kwargs.get('SDnet', '')
        dir3 = osp.join(dir3, f'net{str(SDnet)}')

    if string == 'W':
        path = osp.join(args.prjct_dir, 'weights_results', dir1, dir2, dir3)

    elif string == 'W_AE':
        path = osp.join(args.prjct_dir, 'weights_results', dir1, dir2, method, 'autoencoder')

    elif string == 'I':
        path = osp.join(args.prjct_dir, 'weights_results', dir1, dir2, dir3)

    elif string == 'P':
        path = osp.join(args.prjct_dir, 'weights_results', dir1, dir2)

    else:
        raise ValueError('type of path not recongnized')

    path = osp.join(path, add) if args.exp == 'mthd__tr_samp' else path
    path = osp.join(path, kwargs.get('add_dir')) if kwargs.get('add_dir') else path
    if not os.path.exists(path):
        os.makedirs(path)

    return path

def get_name(method, string, **kwargs):
    """ 
    determine name of file 
    e.g. W=1 in kwargs return name of file of autoencoder weights

    Returns: path to directory
    """
    
    if string == 'W':

        n_sensor = kwargs.get('n_sensor')
        seq_len = kwargs.get('seq_len')
        bottle_neck = kwargs.get('bottle_neck')

        if method == 'are' or method == 'AFE':
            return 'weights_s{}_sq{}_bn{}'.format(n_sensor, seq_len, bottle_neck) 

        elif method == 'pds':
            return 'weights_s{}_bn{}'.format(n_sensor, bottle_neck)

        elif method == 'sd':
            if kwargs.get('drop'):
                return 'weights_s{}_drop'.format(n_sensor)
            else:
                return 'weights_s{}'.format(n_sensor)

        elif method == 'FCNN':
            return 'weights_s{}'.format(n_sensor)

    elif string == 'W_AE':

        bn = kwargs.get('bottle_neck')
        dropout = kwargs.get('drop')
        
        if dropout:
            return 'weights_bn{}_drop{}'.format(bn, dropout)
        else:
            return 'weights_bn{}'.format(bn)



In [6]:
def get_data():
  """ 
  load data from data_dir
  data is strored in args.raw_data
  """
  path_data = args.data_dir

  if hasattr(args, 'raw_data') and hasattr(args, 'last_data_type') and args.data_type == args.last_data_type:
        print('using previously loaded data')

  else:

        start_time = time.time()
        if args.data_type == 'transient':
            print('loading transient flow past cylinder data...')
            td180 = np.load(osp.join(path_data, 'data180.npy'))
            td190 = np.load(osp.join(path_data, 'data190.npy'))
            td200 = np.load(osp.join(path_data, 'data200.npy'))
            td185 = np.load(osp.join(path_data, 'data185.npy'))
            td195 = np.load(osp.join(path_data, 'data195.npy'))
            args.raw_data = (td180, td190, td200, td185, td195)
            print('data loaded')

        elif args.data_type == 'periodic':
            args.raw_data = np.load(osp.join(path_data, 'data190+.npy'))
            print('data loaded')

        elif args.data_type == 'turbulence':
            args.raw_data = np.load(osp.join(path_data, 'tb_128_5000.npy'))
            print('data loaded')
        
        elif args.data_type == 'sea_temp':
            land_sea = np.load(osp.join(path_data, 'SSTdata1990.npz'))['arr_0']

            t = land_sea[0]
            n_pixel = len(t)
            len_data = len(land_sea[:, 0])
            land_pos = np.where(t > 1e+30)[0] 

            full_idx = np.arange(n_pixel)
            args.sea_idx = np.delete(full_idx, land_pos)
            args.raw_data = np.delete(land_sea, [land_pos], 1)
            print('data loaded')

        elif args.data_type == 'burgers1D':
            args.raw_data = np.load(osp.join(path_data, 'run49_step51_tend0.5_nu0.01.npy'))
            print('data loaded')

        else:
            print('data_type is not recognised')

        print("--- %s seconds --" % (time.time() - start_time))  
        args.last_data_type = args.data_type


def img_dims(flat=0):
    """ returns data image dimentions """

    if args.data_type == 'periodic':
        nx, ny = 251, 168
        vec_dim = nx*ny
    if args.data_type == 'transient':
        nx, ny = 502, 252
        vec_dim = nx*ny
    if args.data_type == 'sea_temp':
        nx, ny = 360, 180
        vec_dim = 44219
    if args.data_type == 'burgers1D':
        nx, ny = 128, 1
        vec_dim = nx*ny
    
    if flat: return vec_dim
    return nx, ny

if 0:
    args.data_type = 'sea_temp'
    get_data()
    print(f'data shape: {args.raw_data}')

if 0:
    args.data_type = 'transient'
    args.exp = 'None'
    get_data()

    savePath = dirname(get_path(0, 'P'))
    print(savePath, args.raw_data[0].shape)
    plot_data = args.raw_data[0], list(range(55, 380, 55))
    Plots().transientFlow_dataVisualize(1, savePath, plot_data, figsize=(22, 7))

In [7]:
def process_data(**kwargs):
    """
    Call :func:`get_data` to load data.
    Add noise to data of certain level.
    Split data for training and testing.
    Calculate mean/std of trainAE.

    Vars: W: number of pixels.

    Returns:
        trainAE (ndarray): (numSampTrain, W)
        validAE (ndarray): (numSampValid, W)
        testAE (ndarray): (numSampTest, W)
        stats (ndarray): (2, W) [mean, std]
    """
  
    noisy_data = SNR = kwargs.get('SNR')
    plot, save = kwargs.get('plot'), kwargs.get('save')

    if args.data_type == 'periodic': 

        get_data()
        td = args.raw_data  
        m, n = img_dims()
        tr_samp = args.tr_samp if hasattr(args, 'tr_samp') else 180

        if noisy_data:
            print('in process_data-- noise data', 'SNR: ', SNR)
            
            td_ = np.zeros_like(td)
            for i in range(len(td)): td_[i,:] = awgn(td[i],kwargs.get('SNR',10))

            image, name = td_[tr_samp+60:tr_samp+120,:][39,:], 'im_true_39_SNR{}'.format(SNR)
            if SNR in kwargs.get('plot_SNR', [0]): show_save_image('', image, plot, save, path=get_path('', 'P'), name=name, contour=0) 

        else :
            print('in process_data-- no noise data')
            td_ = td

        trainAE = td_[0:tr_samp,:]
        validAE = td_[tr_samp:tr_samp+60,:]
        testAE = td_[tr_samp+60:tr_samp+120,:]  
 

    elif args.data_type == 'transient':

        get_data()
        td180, td190, td200, td185, td195 = args.raw_data
        tr_samp = len(td180)*3
        args.n_TrainDatasets = 3
        args.n_TestDatasets = 1

        if noisy_data:
            print('in process_data-- noise data', 'SNR =',SNR)
            
            td1_ = td2_ = td3_ = td4_ = td5_ = np.zeros_like(td180)
            for i in range(len(td1_)):
                td1_[i,:] = awgn(td180[i],SNR)
                td2_[i,:] = awgn(td190[i],SNR)
                td3_[i,:] = awgn(td200[i],SNR)
                td4_[i,:] = awgn(td185[i],SNR)
                td5_[i,:] = awgn(td195[i],SNR)
            image, name = td5_[335,:], 'im_true_335_SNR{}'.format(SNR)
            if SNR in kwargs.get('plot_SNR', [0]): show_save_image('', image, plot, save, path=get_path('', 'P'), name=name, contour=0) 

        else :
            print('in process_data-- no noise data')
            td1_, td2_, td3_, td4_, td5_ = td180, td190, td200, td185, td195
        
        trainAE = np.concatenate((td1_, td2_, td3_), axis=0)
        validAE = td4_
        testAE = td5_ 


    elif args.data_type == 'sea_temp':

        get_data()
        td = args.raw_data
        tr_samp = args.tr_samp if hasattr(args, 'tr_samp') else 400

        if noisy_data:
            print('in process_data-- noise data', 'SNR: ', SNR)
            
            td_ = np.zeros_like(td)
            for i in range(len(td)):
                    td_[i,:] = awgn(td[i], SNR)
                
            image, name = td_[tr_samp+100:tr_samp+200,:][0], 'im_true_0_SNR{}'.format(SNR)
            if SNR in kwargs.get('plot_SNR', [0]): show_save_image('', image, plot, save, path=get_path('', 'P'), name=name)

        else :
            print('in process_data-- no noise data')
            td_ = td
        
        trainAE = td_[0:tr_samp+0,:]
        validAE = td_[tr_samp+0:tr_samp+100,:]
        testAE = td_[tr_samp+100:tr_samp+300,:]

    if args.data_type == 'burgers1D': 

        get_data()
        td = args.raw_data  
        m, n = img_dims()

        tr_samp = args.tr_samp if hasattr(args, 'tr_samp') else 51*32
        args.n_TrainDatasets = 32
        args.n_TestDatasets = 4

        if noisy_data:
            print('in process_data-- noise data', 'SNR: ', SNR)
            
            td_ = np.zeros_like(td)
            for i in range(len(td)): td_[i,:] = awgn(td[i],kwargs.get('SNR'))

        else :
            print('in process_data-- no noise data')
            td_ = td

        trainAE = td_[0:tr_samp,:]
        validAE = td_[tr_samp:tr_samp+15,:]
        testAE1 = td_[tr_samp+51*2:tr_samp+51*3,:]
        testAE2 = td_[tr_samp+51*7:tr_samp+51*8,:]
        testAE3 = td_[tr_samp+51*9:tr_samp+51*10,:]
        testAE4 = td_[tr_samp+51*15:tr_samp+51*16,:]
        testAE = np.concatenate((testAE1, testAE2, testAE3, testAE4), axis=0)

  
    AE_mean = np.mean(trainAE, axis = 0)  
    AE_std = np.std(trainAE, axis = 0)  

    if args.data_type == 'sea_temp': 
        AE_std = np.ones_like(AE_std)

    stats = np.array([AE_mean, AE_std])
    stats[stats==0] = 0.0000001
    print('number of training samples:', tr_samp)

    return trainAE, validAE, testAE, stats

In [8]:
def sensor_data(trainAE_stan, validAE_stan, testAE_stan, sensor_num, seed=0):

    n_train, n_valid, n_test = trainAE_stan.shape[0], validAE_stan.shape[0], testAE_stan.shape[0]

    sqrt_s = int(sensor_num**0.5)
    strain_in = (resize(trainAE_stan, (n_train, sqrt_s, sqrt_s))).reshape(n_train, sensor_num)
    svalid_in = (resize(validAE_stan, (n_valid, sqrt_s, sqrt_s))).reshape(n_valid, sensor_num)
    stest_in = (resize(testAE_stan, (n_test, sqrt_s, sqrt_s))).reshape(n_test, sensor_num)

    return strain_in, svalid_in, stest_in


def standardize(a, b, c, stats):

  a_stan = (a - stats[0, :])/stats[1, :]
  b_stan = (b - stats[0, :])/stats[1, :]
  c_stan = (c - stats[0, :])/stats[1, :]
  
  return a_stan, b_stan, c_stan 
  

class Dict2Class(object):
	
	def __init__(self, my_dict):
		for key in my_dict:
			setattr(self, key, my_dict[key])

In [9]:
def sensor_cord_data(sensor_num, seed=0):
    """ 
    Sensor pixel co-ordinates for all data_types.

    Returns:
        s_idx_in_flatten (ndarray, int): (numSensor,) 
        cords (ndarray, int): (numSensor, 2)
    """
    nx, ny = m, n = img_dims()
    print(f'sensor Location seed: {seed}')

    if args.data_type == 'periodic':

        theata = np.linspace(0, 2*np.pi, 300)
        x_cord = np.round(25 * np.cos(theata)) + 0
        y_cord = np.round(25 * np.sin(theata)) + n/2
        cords_ = np.vstack((x_cord,y_cord)).T
        cords_ = np.unique(cords_, axis=0)
        idx = cords_[:,0] > 0
        cords_ = cords_[idx,:]
        # Changing the seed will change the position of 
        # sensors and networks will have to be retrained.
        np.random.seed(3265 + seed)  
        idx = np.random.choice(range(cords_.shape[0]), sensor_num, False)
        cords = np.int64(cords_[idx,:])

        mask = np.zeros((m,n))
        mask[cords[:,0], cords[:,1]] = 1
        s_idx_in_flatten = np.where(mask.reshape(-1) == 1)
        s_idx_in_flatten = np.asarray(s_idx_in_flatten).ravel()

    elif args.data_type == 'transient':

        theata = np.linspace(0, 2*np.pi, 300)
        x_cord = np.round(24 * np.cos(theata)) + 0
        y_cord = np.round(24 * np.sin(theata)) + n/2
        cords_ = np.vstack((x_cord,y_cord)).T
        cords_ = np.unique(cords_, axis=0)
        idx = cords_[:,0] > 0
        cords_ = cords_[idx,:]
        # Changing the seed will change the position of 
        # sensors and networks will have to be retrained.
        np.random.seed(323 + seed)
        idx = np.random.choice(range(cords_.shape[0]), sensor_num, False)
        cords = np.int64(cords_[idx,:])

        mask = np.zeros((m,n))
        mask[cords[:,0], cords[:,1]] = 1
        s_idx_in_flatten = np.where(mask.reshape(-1) == 1)
        s_idx_in_flatten = np.asarray(s_idx_in_flatten).ravel()

    elif args.data_type == 'sea_temp': 

        idices = np.arange(0, 44219)
        # Changing the seed will change the position of 
        # sensors and networks will have to be retrained.
        np.random.seed(73 + seed)
        s_idx_in_flatten = np.random.choice(idices, sensor_num, False)

        flat_sea_mask = np.zeros((44219))
        flat_sea_mask[s_idx_in_flatten] = 1
        flat_mask = np.zeros((m*n))
        for i, j in zip(args.sea_idx, count(0, 1)):
            flat_mask[i] = flat_sea_mask[j]

        mask = flat_mask.reshape(n, m)
        cords = np.where(mask == 1)
        cords = np.asarray(cords)

        # plt.figure()
        # plt.scatter(cords[1], cords[0])
        # plt.show()

    elif args.data_type == 'burgers1D':

        idices = np.arange(0, nx*ny)
        np.random.seed(seed)
        # s_idx_in_flatten = np.random.choice(idices, sensor_num, False)
        s_idx_in_flatten = np.arange(0, nx*ny, round(nx*ny/(sensor_num+1)-0.6))[1:sensor_num+1]
        sensor_mask = np.zeros((nx*ny))
        sensor_mask[s_idx_in_flatten] = 1
        cords = np.where(sensor_mask.reshape(nx, ny) == 1)
        cords = np.asarray(cords)
    pdb.set_trace()
    return s_idx_in_flatten, cords

if 0:
    """ plot sensor locations """
    args.data_type = 'transient'

    for seed in [10, 46, 17, 4, 50]:
        _, sensorCoords = sensor_cord_data(2, seed=seed)

        nx, ny = img_dims()
        imData = np.zeros((nx*ny))

        fig, ax = plt.subplots()
        Plots().cylinderFlow_imshow(imData, ax, cmocean.cm.balance, -1, 1, coords=sensorCoords)

if 0:
    """ plot sensor locations """
    args.data_type = 'burgers1D'

    for seed in [10]:
        _, sensorCoords = sensor_cord_data(10, seed=seed)
        print(sensorCoords)

        nx, ny = img_dims()
        imData = np.zeros((nx*ny))

        fig, ax = plt.subplots()
        Plots().burgers1D_imshow(imData, ax, 0, -1, 1, coords=sensorCoords)

## Plot

In [10]:
class Plots:
    """ Plots data samples by creating seperate object for every data type
    """
    marker = ['-o', '-*', '-,', '-x', '-+', '-P', '-s', '-D', '-p', '-v']

    def save_show(self, plot, save_Path, fig):
        if save_Path:
            fig.savefig(save_Path+'.pdf', format='pdf')
            print(f'saved plot: {save_Path}.pdf')
        fig.show() if plot else 0


    def plot_graph(self, rerror_train, rerror_test, savePath=None):
        fig = plt.figure()
        plt.plot(rerror_train, lw=2, label='Trainings error', color='#377eb8',)  
        plt.plot(rerror_test, lw=2, label='Valid error', color='#e41a1c',)            
        plt.tick_params(axis='x', labelsize=14) 
        plt.tick_params(axis='y', labelsize=14) 
        plt.locator_params(axis='y', nbins=10)
        plt.locator_params(axis='x', nbins=10)
        
        plt.ylabel('Error', fontsize=14)
        plt.xlabel('Epochs', fontsize=14)
        plt.grid(False)
        plt.yscale("log")
        plt.legend(fontsize=14)
        fig.tight_layout()

        self.save_show(1, savePath, fig)


    def cylinderFlow_imshow(self, imData, axes, cmap, v_min, v_max, **kwargs):

        nx, ny = img_dims()
        imD = imData.reshape((nx, ny))

        im = axes.imshow(imD.T, cmap=cmap, interpolation='none', vmin=v_min, vmax=v_max)
        
        if kwargs.get('contour', 1): 
            mX, mY = np.meshgrid(np.arange(0, nx, 1), np.arange(0, ny, 1))
            try: axes.contourf(mX, mY, imD.T, 80, cmap=cmap, alpha=1, vmin=v_min+3, vmax=v_max-3) 
            except: axes.contourf(mX, mY, imD.T, 80, cmap=cmap, alpha=1, vmin=v_min, vmax=v_max) 
        if 'coords' in kwargs.keys():
            coords = kwargs.get('coords')
            axes.scatter(coords[:,0], coords[:,1], marker='.', color='#ff7f00', s=100, zorder=5)
        axes.axis('off')


    def seaTemp_imshow(self, imData, axes, cmap, v_min, v_max, **kwargs):

        nx, ny = img_dims()
        recon = np.ones((nx*ny))*35
        recon[args.sea_idx] = imData
        recon_im = recon.reshape(ny, nx)
        # recon_im = np.flip(recon_im, axis=0)

        axes.imshow(recon_im, cmap=cmap, vmin=v_min, vmax=v_max)
        axes.invert_yaxis()
        if 'coords' in kwargs.keys():
            coords = kwargs.get('coords')
            axes.scatter(coords[1], coords[0], marker='.', color='#ff7f00', s=40, zorder=5)
        axes.axis('off')


    def burgers1D_imshow(self, imData, axes, cmap, v_min, v_max, linestyle='-', label='', **kwargs):

        nx, ny = img_dims()
        imD = imData

        # im = axes.plot(imD)
        axes.plot(imD, label=label, linestyle=linestyle)
        
        if 'coords' in kwargs.keys():
            coords = kwargs.get('coords')
            axes.scatter(coords[0], coords[1], marker='.', color='#ff7f00', s=100, zorder=5)
        # axes.axis('off')

    
    def sst_fig(self, show: bool, savePath: str, plot_data: tuple):

        ARE_dataPred, PDS_dataPred, SD_dataPred, plotParams = plot_data
        idx = plotParams

        mpl.rcParams['font.family'] = ['serif'] # default is sans-serif
        rc('text', usetex=False)
        rc('font', size=8)


        fig, ax = plt.subplots(2, 2, figsize=(10, 6))
        fig.subplots_adjust(wspace=0.1)

        Ar = np.array(ARE_dataPred['pred'])[idx]
        Pd = np.array(PDS_dataPred['pred'])[idx]
        Sd = np.array(SD_dataPred['pred'])[idx]
        Tr = np.array(ARE_dataPred['true'])[idx]

        coords = np.array(ARE_dataPred['coords'])  # sensor co-ordinates
        # assert Tr == PDS_dataPred['true'][idx]
        
        cmap = 'terrain'
        v_max = np.max(np.array([Ar, Pd, Sd, Tr]))
        v_min = np.min(np.array([Ar, Pd, Sd, Tr]))
        # print(f'v_min: {v_min} \n v_max: {v_max}')
        # print(f'{np.max(np.array([Ar]))}, {np.max(np.array([Pd]))}, {np.max(np.array([Sd]))}, {np.max(np.array([Tr]))}')

        self.seaTemp_imshow(Pd, ax[0, 0], cmap, v_min, v_max, coords=coords)
        self.seaTemp_imshow(Ar, ax[0, 1], cmap, v_min, v_max, coords=coords)
        self.seaTemp_imshow(Sd, ax[1, 0], cmap, v_min, v_max, coords=coords)
        self.seaTemp_imshow(Tr, ax[1, 1], cmap, v_min, v_max, coords=coords)

        # ax.set_title('Title with special font: {}'.format(fname), fontproperties = prop, fontsize = 14)
        ax[0, 0].set_title('PDS')
        ax[0, 1].set_title('ARE')
        ax[1, 0].set_title('SD')
        ax[1, 1].set_title('Ground Truth')

        # ---------------- color bar
        p0 = ax[0, 1].get_position().get_points().flatten()
        p1 = ax[1, 1].get_position().get_points().flatten()
        ax_cbar = fig.add_axes([p1[2]+0.0075, p1[1], 0.01, p0[3]-p1[1]])
        ticks = np.linspace(0, 1, 5)
        tickLabels = np.linspace(v_min, v_max, 5)
        tickLabels = ["{:02.2f}".format(t0) for t0 in tickLabels]
        cbar = mpl.colorbar.ColorbarBase(ax_cbar, cmap=plt.get_cmap(cmap), orientation='vertical', ticks=ticks)
        cbar.set_ticklabels(tickLabels)

        self.save_show(show, savePath, fig)


    def periodicFlow_fig(self, show: bool, savePath: str, plot_data: tuple, figsize=(9, 6)):

        ARE_dataPred, PDS_dataPred, SD_dataPred, plotParams = plot_data
        idx = plotParams

        mpl.rcParams['font.family'] = ['serif'] # default is sans-serif
        rc('text', usetex=False)
        rc('font', size=8)


        fig, ax = plt.subplots(2, 2, figsize=figsize)
        fig.subplots_adjust(wspace=0.01)

        Ar = np.array(ARE_dataPred['pred'])[idx]
        Pd = np.array(PDS_dataPred['pred'])[idx]
        Sd = np.array(SD_dataPred['pred'])[idx]
        Tr = np.array(ARE_dataPred['true'])[idx]

        coords = np.array(ARE_dataPred['coords'])  # sensor co-ordinates
        # assert Tr == PDS_dataPred['true'][idx]
        
        cmap = cmocean.cm.balance
        v_max = np.max(np.abs(np.array([Ar, Pd, Sd, Tr]))) * 0.65
        v_min = -v_max
        print(f'v_min: {v_min} \n v_max: {v_max}')

        self.cylinderFlow_imshow(Pd, ax[0, 0], cmap, v_min, v_max, coords=coords)
        self.cylinderFlow_imshow(Ar, ax[0, 1], cmap, v_min, v_max, coords=coords)
        # self.cylinderFlow_imshow(Ar, ax[0, 1], cmap, np.min(Ar), np.max(Ar), coords=coords)
        self.cylinderFlow_imshow(Sd, ax[1, 0], cmap, v_min, v_max, coords=coords)
        self.cylinderFlow_imshow(Tr, ax[1, 1], cmap, v_min, v_max, coords=coords)

        # ax.set_title('Title with special font: {}'.format(fname), fontproperties = prop, fontsize = 14)
        ax[0, 0].set_title('PDS')
        ax[0, 1].set_title('ARE')
        ax[1, 0].set_title('SD')
        ax[1, 1].set_title('Ground Truth')

        # ---------------- color bar
        p0 = ax[0, 1].get_position().get_points().flatten()
        p1 = ax[1, 1].get_position().get_points().flatten()
        ax_cbar = fig.add_axes([p1[2]+0.0075, p1[1], 0.01, p0[3]-p1[1]])
        ticks = np.linspace(0, 1, 5)
        tickLabels = np.linspace(v_min, v_max, 5)
        tickLabels = ["{:02.2f}".format(t0) for t0 in tickLabels]
        cbar = mpl.colorbar.ColorbarBase(ax_cbar, cmap=plt.get_cmap(cmap), orientation='vertical', ticks=ticks)
        cbar.set_ticklabels(tickLabels)

        self.save_show(show, savePath, fig)


    def transientFlow_graphPlot_senLoc(self, show, savePath, plot_data, plotParams):
        
        keys = list(plot_data.keys())
        data = lambda i: plot_data[keys[i]]

        fig, ax = plt.subplots()

        for i in range(len(plot_data)):
            xx = range(len(data(i)['x_ticks_lables']))
            ax.plot(xx, data(i)['data'], self.marker[i], label=data(i)['legend'], color=data(i)['color'])
        ax.legend()
        ax.set_xticks(range(len(data(i)['x_ticks_lables'])))
        ax.set_xticklabels(data(0)['x_ticks_lables'])

        ax.set_xlabel(plotParams['xlabel'], fontsize=14)
        ax.set_ylabel(plotParams['ylabel'], fontsize=14)

        self.save_show(show, savePath, fig)


    def transientFlow_imgPlot_senLoc(self, show, savePath, plot_data, figsize=(18, 9)):

        ARE_dataPred, plotParams = plot_data
        idx = plotParams

        mpl.rcParams['font.family'] = ['serif'] # default is sans-serif
        rc('text', usetex=False)
        rc('font', size=8)

        fig, ax = plt.subplots(3, 2, figsize=figsize)
        fig.subplots_adjust(wspace=0.001)

        Ar = []
        for i in range(5):
            Ar.append(np.array(ARE_dataPred[i]['pred'])[idx])
        Tr = np.array(ARE_dataPred[0]['true'])[idx]

        sensorCoords = lambda i: np.array(ARE_dataPred[i]['coords'])  # sensor co-ordinates
        # assert Tr == PDS_dataPred['true'][idx]
        
        cmap = cmocean.cm.balance
        v_max = np.max(np.abs(np.array([Ar[0], Ar[1], Ar[2], Ar[3], Ar[4], Tr]))) * 0.65
        v_min = -v_max
        print(f'v_min: {v_min} \n v_max: {v_max}')

        self.cylinderFlow_imshow(Tr, ax[0, 0], cmap, v_min, v_max)
        self.cylinderFlow_imshow(Ar[0], ax[0, 1], cmap, v_min, v_max, coords=sensorCoords(0))
        self.cylinderFlow_imshow(Ar[1], ax[1, 0], cmap, v_min, v_max, coords=sensorCoords(1))
        self.cylinderFlow_imshow(Ar[2], ax[1, 1], cmap, v_min, v_max, coords=sensorCoords(2))
        self.cylinderFlow_imshow(Ar[3], ax[2, 0], cmap, v_min, v_max, coords=sensorCoords(3))
        self.cylinderFlow_imshow(Ar[4], ax[2, 1], cmap, v_min, v_max, coords=sensorCoords(4))

        # ax.set_title('Title with special font: {}'.format(fname), fontproperties = prop, fontsize = 14)
        ax[0, 0].set_title('Ground Truth')
        ax[0, 1].set_title('ARE1')
        ax[1, 0].set_title('ARE2')
        ax[1, 1].set_title('ARE3')
        ax[2, 0].set_title('ARE4')
        ax[2, 1].set_title('ARE5')

        # ---------------- color bar
        p0 = ax[0, 1].get_position().get_points().flatten()
        p1 = ax[2, 1].get_position().get_points().flatten()
        ax_cbar = fig.add_axes([p1[2]+0.009, p1[1], 0.01, p0[3]-p1[1]])
        ticks = np.linspace(0, 1, 5)
        tickLabels = np.linspace(v_min, v_max, 5)
        tickLabels = ["{:02.2f}".format(t0) for t0 in tickLabels]
        cbar = mpl.colorbar.ColorbarBase(ax_cbar, cmap=plt.get_cmap(cmap), orientation='vertical', ticks=ticks)
        cbar.set_ticklabels(tickLabels)

        self.save_show(show, savePath, fig)

    
    def transientFlow_imgPlot_mthd2(self, show, savePath, plot_data, figsize=(9, 9)):

        AFE_dataPred, FCNN_dataPred, plotParams = plot_data
        idx = plotParams

        mpl.rcParams['font.family'] = ['serif'] # default is sans-serif
        rc('text', usetex=False)
        rc('font', size=8)


        fig, ax = plt.subplots(2, 2, figsize=figsize)
        fig.subplots_adjust(wspace=0.01)

        Af = np.array(AFE_dataPred['pred'])[idx]
        Fc = np.array(FCNN_dataPred['pred'])[idx]
        Tr = np.array(AFE_dataPred['true'])[idx]

        coords = np.array(AFE_dataPred['coords'])  # sensor co-ordinates
        # assert Tr == PDS_dataPred['true'][idx]
        
        cmap = cmocean.cm.balance
        v_max = np.max(np.abs(np.array([Af, Fc, Tr]))) * 0.65
        v_min = -v_max
        print(f'v_min: {v_min} \n v_max: {v_max}')

        self.cylinderFlow_imshow(Tr, ax[0, 0], cmap, v_min, v_max, coords=coords)
        self.cylinderFlow_imshow(Af, ax[0, 1], cmap, v_min, v_max, coords=coords)
        self.cylinderFlow_imshow(Fc, ax[1, 0], cmap, v_min, v_max, coords=coords)

        # ax.set_title('Title with special font: {}'.format(fname), fontproperties = prop, fontsize = 14)
        ax[0, 0].set_title('Ground Truth')
        ax[0, 1].set_title('AFE')
        ax[1, 0].set_title('FCNN')

        # ---------------- color bar
        p0 = ax[0, 1].get_position().get_points().flatten()
        p1 = ax[1, 1].get_position().get_points().flatten()
        ax_cbar = fig.add_axes([p1[2]+0.0075, p1[1], 0.01, p0[3]-p1[1]])
        ticks = np.linspace(0, 1, 5)
        tickLabels = np.linspace(v_min, v_max, 5)
        tickLabels = ["{:02.2f}".format(t0) for t0 in tickLabels]
        cbar = mpl.colorbar.ColorbarBase(ax_cbar, cmap=plt.get_cmap(cmap), orientation='vertical', ticks=ticks)
        cbar.set_ticklabels(tickLabels)

        self.save_show(show, savePath, fig)


    def transientFlow_dataVisualize(self, show, savePath, plot_data, figsize=(18, 9)):

        flowData, idx_ls = plot_data
        flowData = flowData[idx_ls]

        mpl.rcParams['font.family'] = ['serif'] # default is sans-serif
        rc('text', usetex=False)
        rc('font', size=14)

        fig, ax = plt.subplots(2, 3, figsize=figsize)
        # fig.subplots_adjust()
        fig.subplots_adjust(wspace=0.01, hspace=0.01)
        
        cmap = cmocean.cm.balance
        v_max = np.max(np.abs(np.array([flowData]))) * 0.65
        v_min = -v_max
        print(f'v_min: {v_min} \n v_max: {v_max}')

        for i in range(3):
            self.cylinderFlow_imshow(flowData[i], ax[0, i], cmap, v_min, v_max)
            self.cylinderFlow_imshow(flowData[i+3], ax[1, i], cmap, v_min, v_max)

        # ---------------- color bar
        p0 = ax[0, 2].get_position().get_points().flatten()
        p1 = ax[1, 2].get_position().get_points().flatten()
        ax_cbar = fig.add_axes([p1[2]+0.009, p1[1], 0.01, p0[3]-p1[1]])
        ticks = np.linspace(0, 1, 5)
        tickLabels = np.linspace(v_min, v_max, 5)
        tickLabels = ["{:02.2f}".format(t0) for t0 in tickLabels]
        cbar = mpl.colorbar.ColorbarBase(ax_cbar, cmap=plt.get_cmap(cmap), orientation='vertical', ticks=ticks)
        cbar.set_ticklabels(tickLabels)

        self.save_show(show, savePath, fig)


    def burgers1D_imgPlot1(self, show, savePath, plot_data, figsize=(18, 9)):

        ARE_predData, PDS_predData, SD_predData, plotParams = plot_data
        idx_ls = plotParams

        mpl.rcParams['font.family'] = ['serif'] # default is sans-serif
        rc('text', usetex=False)
        rc('font', size=9)

        fig, ax = plt.subplots(2, 2, figsize=figsize)
        fig.subplots_adjust(hspace=0.001, wspace=0.1)

        Ar = []; Pd = []; Sd = []; Tr = []
        for idx in idx_ls:
            Ar.append(np.array(ARE_predData['pred'])[idx])
            Pd.append(np.array(PDS_predData['pred'])[idx])
            Sd.append(np.array(SD_predData['pred'])[idx])
            Tr.append(np.array(PDS_predData['true'])[idx])

        sensorCoords = lambda i: np.array(ARE_predData['coords'][i])  # sensor co-ordinates
        # assert Tr == PDS_dataPred['true'][idx]
        
        cmap = 0
        v_max = 0
        v_min = 0
        print(f'v_min: {v_min} \n v_max: {v_max}')

        ax_ls = [ax[0, 0], ax[0, 1], ax[1, 0], ax[1, 1]]
        for i, idx in enumerate(idx_ls):
            # pdb.set_trace()
            self.burgers1D_imshow(Tr[i], ax_ls[i], cmap, v_min, v_max, linestyle='-', label='True')
            self.burgers1D_imshow(Ar[i], ax_ls[i], cmap, v_min, v_max, linestyle='--', label='ARE')
            self.burgers1D_imshow(Pd[i], ax_ls[i], cmap, v_min, v_max, linestyle='-.', label='PDS')
            self.burgers1D_imshow(Sd[i], ax_ls[i], cmap, v_min, v_max, linestyle=':', label='SD')
            ax_ls[i].legend(fontsize=10)

        self.save_show(show, savePath, fig)


    def burgers1D_imgPlot(self, show, savePath, plot_data, figsize=(18, 9)):

        ARE_predData, PDS_predData, SD_predData, plotParams = plot_data
        idx = plotParams

        mpl.rcParams['font.family'] = ['serif'] # default is sans-serif
        rc('text', usetex=False)
        rc('font', size=9)

        fig, ax = plt.subplots(2, 2, figsize=figsize)
        fig.subplots_adjust(hspace=0.001, wspace=0.1)

        Ar = []; Pd = []; Sd = []; Tr = []
        Ar.append(np.array(ARE_predData['pred'])[idx])
        Pd.append(np.array(PDS_predData['pred'])[idx])
        Sd.append(np.array(SD_predData['pred'])[idx])
        Tr.append(np.array(PDS_predData['true'])[idx])

        sensorCoords = lambda i: np.array(ARE_predData['coords'])  # sensor co-ordinates
        # assert Tr == PDS_dataPred['true'][idx]
        
        cmap = 0
        v_max = 0
        v_min = 0
        print(f'v_min: {v_min} \n v_max: {v_max}')

        ax_ls = [ax[0, 0], ax[0, 1], ax[1, 0], ax[1, 1]]
        # self.burgers1D_imshow(Tr[0], ax_ls[0], cmap, v_min, v_max, label='True', coords=sensorCoords(idx))
        # self.burgers1D_imshow(Ar[0], ax_ls[1], cmap, v_min, v_max, label='ARE', coords=sensorCoords(idx))
        # self.burgers1D_imshow(Pd[0], ax_ls[2], cmap, v_min, v_max, label='PDS', coords=sensorCoords(idx))
        # self.burgers1D_imshow(Sd[0], ax_ls[3], cmap, v_min, v_max, label='SD', coords=sensorCoords(idx))
        self.burgers1D_imshow(Tr[0], ax_ls[0], cmap, v_min, v_max, label='True')
        self.burgers1D_imshow(Ar[0], ax_ls[1], cmap, v_min, v_max, label='ARE')
        self.burgers1D_imshow(Pd[0], ax_ls[2], cmap, v_min, v_max, label='PDS')
        self.burgers1D_imshow(Sd[0], ax_ls[3], cmap, v_min, v_max, label='SD')
        
        ymin = np.min(Tr[0])*(1+0.2)
        ymax = np.max(Tr[0])*(1+0.2)
        for i in range(4):
            ax_ls[i].legend(fontsize=12)
            ax_ls[i].set_ylim([ymin, ymax])

        self.save_show(show, savePath, fig)




In [11]:

def show_save_image(method, images, plot, save, **kwargs):

  istupl = True if isinstance(images, tuple) else False

  len_tuple = len(images) if istupl else 1
  get_value = lambda index, tuple_ : tuple_[index] if istupl else tuple_
  i = kwargs.get('i', 0)
  stats = kwargs.get('stats', np.array([0]))

  if istupl:
    assert len_tuple == len(save) == len(plot)
    if any(save):
      assert len(kwargs.get('name')) == len_tuple, 'mention all names of images for saving'
  else:
    if save: assert kwargs.get('name') 

  for idx in range(len_tuple):
      
        save_ = get_value(idx, save)
        plot_ = get_value(idx, plot)
        name = get_value(idx, kwargs.get('name', ['im']*len_tuple))

        sensorCoords = cords = kwargs.get('cords', np.array([0]))

        t = get_value(idx, images)
        t = t.cpu().data.numpy() if torch.is_tensor(t) else t
        if t.ndim == 1:
            t = t
        elif t.ndim == 2:
            t = t[-1]
        elif t.ndim == 3:
            t = t[-1, -1]
        t = t * stats[1, :] + stats[0, :] if stats.any() else t

        ny, nx = m, n = img_dims()

        if args.data_type == 'periodic' or args.data_type == 'transient':
            minmax = np.max(np.abs(t)) * 0.65
            imData = t

            fig, ax = plt.subplots(figsize=(7.9,4.7))
            if cords.any(): Plots().cylinderFlow_imshow(imData, ax, cmocean.cm.balance, -minmax, minmax, coords=sensorCoords)
            else: Plots().cylinderFlow_imshow(imData, ax, cmocean.cm.balance, -minmax, minmax)

        if args.data_type == 'turbulence':

            t_ = t.reshape((ny, nx))
            plt.figure(facecolor="white",  edgecolor='k')
            im = plt.imshow(t_)
      
        if args.data_type == 'sea_temp':

            v_min = np.min(t); v_max = np.max(t); imData = t

            fig, ax = plt.subplots()
            if cords.any(): Plots().seaTemp_imshow(imData, ax, 'terrain', v_min, v_max, coords=sensorCoords)
            else: Plots().seaTemp_imshow(imData, ax, 'terrain', v_min, v_max)

        if args.data_type == 'burgers1D':

            v_min = np.min(t); v_max = np.max(t); imData = t

            fig, ax = plt.subplots()
            if cords.any(): Plots().burgers1D_imshow(imData, ax, 'terrain', v_min, v_max, coords=sensorCoords)
            else: Plots().burgers1D_imshow(imData, ax, 'terrain', v_min, v_max)

        if save_:
            path = kwargs.get('path') if kwargs.get('path') else get_path(method, 'I', seed = kwargs.get('seed', 0)) 
            fig.savefig(osp.join(path, '{}.pdf'.format(name)), format='pdf')
            print('SAVED ==', osp.join(path, '{}.pdf'.format(name)))
        
        fig.show() if plot_ else 0
        

## Add noise

In [12]:

def awgn(s,SNRdB,L=1):
    """
    AWGN channel
    Add AWGN noise to input signal. The function adds AWGN noise vector to signal
    's' to generate a resulting signal vector 'r' of specified SNR in dB. It also
    returns the noise vector 'n' that is added to the signal 's' and the power 
    spectral density N0 of noise added
    Parameters:
        s : input/transmitted signal vector
        SNRdB : desired signal to noise ratio (expressed in dB) for the received signal
        L : oversampling factor (applicable for waveform simulation) default L = 1.
    Returns:
        r : received signal vector (r=s+n)
"""
    gamma = 10**(SNRdB/10) #SNR to linear scale
    if s.ndim==1:# if s is single dimensional vector
        P=L*sum(abs(s)**2)/len(s) #Actual power in the vector
    else: # multi-dimensional signals like MFSK
        P=L*sum(sum(abs(s)**2))/len(s) # if s is a matrix [MxN]
    N0=P/gamma # Find the noise spectral density
    if isrealobj(s):# check if input is real/complex object type
        n = sqrt(N0/2)*standard_normal(s.shape) # computed noise
    else:
        n = sqrt(N0/2)*(standard_normal(s.shape)+1j*standard_normal(s.shape))
    r = s + n # received signal
    return r

# td_n2 = np.zeros((300,42168))
# # td_n2 = awgn(td,10)
# for i in range(len(td)):
#   td_n2[i,:] = awgn(td[i],1)


# AUTO RECURRENT ESTIMATION (ARE)

## Autoencoder data

In [13]:
class sensorgcdatasetAE(Dataset):

    def __init__(self, in_file, stats, transform=None):
        self.sensor_frame = in_file
        self.stats = stats
        self.transform = transform

    def __len__(self):
        return len(self.sensor_frame)

    def __getitem__(self, idx):
        sensor = self.sensor_frame[idx, :]
        sensor = (sensor - self.stats[0, :]) / self.stats[1, :]
        sensor = torch.from_numpy(sensor).float()
        return sensor

## Autoencoder Net

In [14]:
class autoencoder(nn.Module):
    def __init__(self, bottle_neck, **kwargs):
        super(autoencoder, self).__init__()
        self.bottle_neck = bottle_neck
        m, n = img_dims()
        k = m*n
        drop = kwargs.get('drop')

        if args.data_type == 'transient':

            if drop:
                print('autoencoder oprating with dropout:', drop)
                self.encoder = nn.Sequential(
                    # nn.Dropout(p =0.5),
                    nn.Linear(k, 2000),
                    nn.BatchNorm1d(num_features = 2000),
                    nn.ReLU(True),
                    # nn.Dropout(p =0.5),
                    nn.Linear(2000, 300),
                    nn.BatchNorm1d(num_features =300),
                    nn.ReLU(True),
                    nn.Dropout(p=drop),
                    nn.Linear(300, bottle_neck))
                self.decoder = nn.Sequential(
                    nn.Linear(bottle_neck, 300),
                    nn.ReLU(True),
                    nn.Linear(300, 2000),       
                    nn.ReLU(True),
                    nn.Linear(2000, k)) 

            else:
                print('autoencoder oprating without dropout layer')
                self.encoder = nn.Sequential(
                    nn.Linear(k, 2000),
                    nn.ReLU(True),
                    nn.Linear(2000, 300), 
                    nn.ReLU(True),
                    nn.Linear(300, bottle_neck)) 
                self.decoder = nn.Sequential(
                    nn.Linear(bottle_neck, 300),
                    nn.ReLU(True),
                    nn.Linear(300, 2000),
                    nn.ReLU(True),
                    nn.Linear(2000, k)) 

        elif args.data_type == 'periodic':

            if drop:
                print('autoencoder oprating with dropout:', drop)
                self.encoder = nn.Sequential(
                    nn.Linear(k, 1024),
                    nn.BatchNorm1d(num_features = 1024),
                    nn.ReLU(True),
                    nn.Linear(1024, 256),
                    nn.BatchNorm1d(num_features =256), 
                    nn.ReLU(True),
                    nn.Dropout(p=drop),
                    nn.Linear(256, bottle_neck)) 
                self.decoder = nn.Sequential(
                    nn.Linear(bottle_neck, 256),
                    nn.ReLU(True),
                    nn.Linear(256, 1024),          
                    nn.ReLU(True),
                    nn.Linear(1024, k)) 
            else:
                print('autoencoder oprating without dropout layer')
                self.encoder = nn.Sequential(
                    nn.Linear(k, 1024),
                    nn.ReLU(True),
                    nn.Linear(1024, 256),
                    nn.ReLU(True),
                    nn.Linear(256, bottle_neck)) 
                self.decoder = nn.Sequential(
                    nn.Linear(bottle_neck, 256),
                    nn.ReLU(True),
                    nn.Linear(256, 1024),
                    nn.ReLU(True),
                    nn.Linear(1024, k)) 
              
        elif args.data_type == 'sea_temp':
            k = 44219

            if drop:
                print('autoencoder oprating with dropout:', drop)
                self.encoder = nn.Sequential(
                    nn.Linear(k, 512),
                    nn.BatchNorm1d(num_features=512),
                    nn.ReLU(True),
                    nn.Linear(512, 256),
                    nn.BatchNorm1d(num_features=256), 
                    nn.ReLU(True),
                    nn.Dropout(p=drop),
                    nn.Linear(256, bottle_neck)) 
                self.decoder = nn.Sequential(
                    nn.Linear(bottle_neck, 256),
                    nn.ReLU(True),
                    nn.Linear(256, 512),          
                    nn.ReLU(True),
                    nn.Linear(512, k)) 
            else:
                print('autoencoder oprating without dropout layer')
                self.encoder = nn.Sequential(
                    nn.Linear(k, 512),
                    # nn.BatchNorm1d(num_features = 512),
                    nn.ReLU(True),
                    nn.Linear(512, 256),
                    # nn.BatchNorm1d(num_features =256), 
                    nn.ReLU(True),
                    nn.Linear(256, bottle_neck)) 
                self.decoder = nn.Sequential(
                    nn.Linear(bottle_neck, 256),
                    # nn.BatchNorm1d(num_features =256),
                    nn.ReLU(True),
                    nn.Linear(256, 512),
                    # nn.BatchNorm1d(num_features =512),
                    nn.ReLU(True),
                    nn.Linear(512, k)) 
            
        if args.data_type == 'burgers1D':

            if drop:
                print('autoencoder oprating with dropout:', drop)
                self.encoder = nn.Sequential(
                    nn.Linear(k, 128),
                    nn.BatchNorm1d(num_features = 128),
                    nn.ReLU(True),
                    nn.Linear(128, 128),
                    nn.BatchNorm1d(num_features =128),
                    nn.ReLU(True),
                    nn.Dropout(p=drop),
                    nn.Linear(128, bottle_neck))
                self.decoder = nn.Sequential(
                    nn.Linear(bottle_neck, 128),
                    nn.ReLU(True),
                    nn.Linear(128, 128),       
                    nn.ReLU(True),
                    nn.Linear(128, k)) 

            else:
                print('autoencoder oprating without dropout layer')
                self.encoder = nn.Sequential(
                    nn.Linear(k, 128),
                    nn.ReLU(True),
                    nn.Linear(128, 128),
                    nn.ReLU(True),
                    nn.Linear(128, bottle_neck)) 
                self.decoder = nn.Sequential(
                    nn.Linear(bottle_neck, 128),
                    nn.ReLU(True),
                    nn.Linear(128, 128),
                    nn.ReLU(True),
                    nn.Linear(128, k)) 
            

    def forward(self, x):
      if x.shape[-1] == self.bottle_neck:
        w = self.decoder(x)
        return w
      else:
        y = self.encoder(x)
        z = self.decoder(y)
        return z,y
      # pdb.set_trace()
      # try :
      #   w = self.decoder(x)
      #   pdb.set_trace()
      #   return w
      # except :
      #   y = self.encoder(x)
      #   z = self.decoder(y)
      #   pdb.set_trace()
      #   return z,y
        

## AUTOENCODER Train Loop

In [15]:
def AutoEcoder(bn, lr, num_epochs, bs, bsv, early_stop, **kwargs):

    bottle_neck = bn
    
    trainAE, validAE, _, stats = process_data(drop=0, tr_samp=kwargs.get('tr_samp'))
    train = sensorgcdatasetAE(trainAE, stats)
    valid = sensorgcdatasetAE(validAE, stats)

    train_data_gen = DataLoader(train, shuffle=True, batch_size=bs)
    valid_data_gen = DataLoader(valid, batch_size=bsv)

    dataloaders = {'train': train_data_gen, 'valid':valid_data_gen}
    dataset_sizes = {'train': len(train_data_gen.dataset), 'valid': len(valid_data_gen.dataset)}

    model = autoencoder(bottle_neck, drop=kwargs.get('drop',None)).to(device)
    criterion = nn.MSELoss().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr,  weight_decay=1e-5)

    prev_valid_loss = prev_train_loss = 100  # anylarge number
    stop=0; ii = 0; k=0
    rerror_train = []
    rerror_valid = []
    
    for epoch in range(num_epochs):
        for phase in ['train']:
            running_loss=0
            if phase == 'train': model.train()
            else:model.eval()

            for data in enumerate(dataloaders[phase]):
                img = data[1]
                img = Variable(img).to(device)
                output, waste = model(img)
                loss = criterion(output, img)
                optimizer.zero_grad()

                if phase=='train':
                    loss.backward()
                    optimizer.step()

                running_loss += output.shape[0]*loss.data

            if phase == 'train':
                train_epoch_loss = running_loss/dataset_sizes[phase]
                train_out_last = output 
                train_in_last = img 
            elif phase == 'valid':
                valid_epoch_loss = running_loss/dataset_sizes[phase]
        stop = stop+1

        progress = f'   ({epoch}) Training loss: {train_epoch_loss:.8f}'
        if epoch%50 == 0: print(progress)
        
        if (train_epoch_loss < prev_train_loss):

            model_wts = model.state_dict()
            prev_train_loss = train_epoch_loss
            stop = 0
            best_train_out, best_train_in = train_out_last, train_in_last
            best_valid_out, best_valid_in = output, img

        if stop == early_stop:
            print('Early stopping criteria fulfilled')
            break

        rerror_train.append(train_epoch_loss)

    images = (best_train_out, best_train_in, best_train_out, best_train_in)
    show, save, path = (1, 1, 1, 1), (1, 1, 1, 1), get_path('are', 'W_AE') 
    name = ('best_train_out', 'best_train_in', 'best_valid_out', 'best_valid_in')
    show_save_image('are', images, show, save, stats=stats, path=path, name=name)

    path_to_weights = get_path('are', 'W_AE')
    weights_name = get_name('', 'W_AE', drop=kwargs.get('drop'), bottle_neck=bottle_neck)
    torch.save(model_wts, osp.join(path_to_weights, weights_name))
    print('==========weights saved==========')

    Plots().plot_graph(rerror_train, rerror_train, savePath=osp.join(path_to_weights, weights_name))

## ARE_train_data

In [16]:
class sensorgcdatasetRNN(Dataset):

    def __init__(self, in_file, out_file, transform=None):
        self.sensor_frame = in_file
        self.gc_frame = out_file
        self.transform = transform

    def __len__(self):
        return len(self.sensor_frame)

    def __getitem__(self, idx):
        sensor = self.sensor_frame[idx, :]
        gc = self.gc_frame[idx, :]
        sensor = torch.from_numpy(sensor).float()
        gc = torch.from_numpy(gc).float()
        return sensor, gc

class sensorgcdatasetAA(Dataset):
    def __init__(self, in_file, out_file, stats, transform=None):
        self.sensor_frame = in_file
        self.gc_frame = out_file
        self.stats = stats
        self.transform = transform

    def __len__(self):
        return len(self.sensor_frame)

    def __getitem__(self, idx):
        sensor = self.sensor_frame[idx, :]
        gc = self.gc_frame[idx, :]
        gc = (gc - self.stats[0, :])/self.stats[1, :]
        sensor = torch.from_numpy(sensor).float()
        gc = torch.from_numpy(gc).float()
        return sensor, gc

def ARE_train_data(sensor_num, bottle_neck, seq_len, **kwargs):
    seq_impct = int((seq_len-1)/2)

    trainAE, validAE, testAE, stats = process_data(SNR=kwargs.get('SNR'), plot=kwargs.get('plot'), save=kwargs.get('save'),
                                                            tr_samp=kwargs.get('tr_samp'), plot_SNR=kwargs.get('plot_SNR'))
    
    len_train_and_valid = len(trainAE[:,0])+ len(validAE[:,0])

    trainAE_stan, validAE_stan, testAE_stan = standardize(trainAE, validAE, testAE, stats)
    strain_in, svalid_in, stest_in = sensor_data(trainAE_stan, validAE_stan, testAE_stan, sensor_num, seed=kwargs.get('seed', 0))
    n_train, n_valid, n_test = trainAE_stan.shape[0], validAE_stan.shape[0], testAE_stan.shape[0]
    assert n_train == len(trainAE[:,0])
    
    # ----------------------------------------------------------------------------
    total = np.concatenate((trainAE, validAE, testAE), axis=0)
    train = sensorgcdatasetAE(total, stats)

    path_to_weights = get_path('are', 'W_AE')
    weights_name = get_name('are', 'W_AE', drop=kwargs.get('drop'), bottle_neck=bottle_neck)
    pretrained_weightsAE = osp.join(path_to_weights, weights_name)
    # pretrained_weightsAE = '/content/drive/MyDrive/Colab Notebooks/are/weights_results/transient/mthd__snsr/are/autoencoder/weights_bn25'

    train_data_gen = DataLoader(train, shuffle=False, batch_size=len(total[:,0]))
    model = autoencoder(bottle_neck, drop=kwargs.get('drop')).to(device)
    model.load_state_dict(torch.load(pretrained_weightsAE, map_location=torch.device(device)))
    model.eval()

    for data in enumerate(train_data_gen):
        # img = data[1]
        # img = Variable(img).to(device)
        waste, gg = model(data[1].to(device))
        gtotal_out = gg.cpu().data.numpy()

    gstats = np.array([np.mean(gtotal_out[0:n_train,:],axis = 0), np.std(gtotal_out[0:n_train,:], axis = 0)])
    del(trainAE,validAE,testAE)
    
    gtotal = (gtotal_out - gstats[0]) / gstats[1] 

    if args.operation_mode == 'online':
            gtrain_out = gtotal[seq_impct:n_train, :] 
            gvalid_out = gtotal[n_train+seq_impct: len_train_and_valid,:]
            gtest = gtotal[len_train_and_valid+seq_impct : len(total[:,0]),:]

    elif args.operation_mode == 'offline':
            gtrain_out = gtotal[seq_impct:n_train - seq_impct, :] 
            gvalid_out = gtotal[n_train+seq_impct: len_train_and_valid-seq_impct,:]
            gtest = gtotal[len_train_and_valid+seq_impct : len(total[:,0])-seq_impct,:]

    # ----------------------------------------------------------------------------
    del_TrainRows_ls = delStates_BW_datasets(n_train, seq_impct, args.n_TrainDatasets)
    gtrain_out = np.delete(gtrain_out, del_TrainRows_ls, 0)

    del_TestRows_ls = delStates_BW_datasets(n_test, seq_impct, args.n_TestDatasets)
    gtest = np.delete(gtest, del_TestRows_ls, 0)

    del(total,train,gtotal)

    # ----------------------------------------------------------------------------
    if args.operation_mode == 'online':
        
        strain_in_stacked = np.zeros((n_train-seq_impct,(seq_impct+1)*sensor_num))
        svalid_in_stacked = np.zeros((n_valid-seq_impct,(seq_impct+1)*sensor_num))
        stest_in_stacked = np.zeros((n_test-seq_impct,(seq_impct+1)*sensor_num))
        
        for i in range(0,n_train-seq_impct):
            strain_in_stacked[i] = np.concatenate([(strain_in[i+sl,:]) for sl in range(seq_impct+1)], axis=None)
        for i in range(0,n_valid-seq_impct):
            svalid_in_stacked[i] = np.concatenate([(svalid_in[i+sl,:]) for sl in range(seq_impct+1)], axis=None)
        for i in range(0,n_test-seq_impct):
            stest_in_stacked[i] = np.concatenate([(stest_in[i+sl,:]) for sl in range(seq_impct+1)], axis=None)
        
        if args.data_type == 'transient' or args.data_type == 'burgers1D':
            strain_in_stacked = np.delete(strain_in_stacked, del_TrainRows_ls-seq_impct, 0)
            stest_in_stacked = np.delete(stest_in_stacked, del_TestRows_ls-seq_impct, 0)


    elif args.operation_mode == 'offline':

        strain_in_stacked = np.zeros((n_train-2*seq_impct,seq_len*sensor_num))
        svalid_in_stacked = np.zeros((n_valid-2*seq_impct,seq_len*sensor_num))
        stest_in_stacked = np.zeros((n_test-2*seq_impct,seq_len*sensor_num))

        for i in range(0,n_train-2*seq_impct) :
            strain_in_stacked[i] = np.concatenate([(strain_in[i+sl,:]) for sl in range(seq_len)], axis=None)
        for i in range(0,n_valid-2*seq_impct) :
            svalid_in_stacked[i] = np.concatenate([(svalid_in[i+sl,:]) for sl in range(seq_len)], axis=None)
        for i in range(0,n_test-2*seq_impct) :
            stest_in_stacked[i] = np.concatenate([(stest_in[i+sl,:]) for sl in range(seq_len)], axis=None)
        
        if args.data_type == 'transient' or args.data_type == 'burgers1D':
            strain_in_stacked = np.delete(strain_in_stacked, del_TrainRows_ls-seq_impct, 0) 
            stest_in_stacked = np.delete(stest_in_stacked, del_TestRows_ls-seq_impct, 0)
    
    return gtrain_out, gvalid_out, strain_in_stacked, svalid_in_stacked, gtest, stest_in_stacked, gstats, stats


def delStates_BW_datasets(n_samp, seq_impct, n_datasets):
    """ Deleting rows in concatenated data at junction of datasets.
        e.g. 3 datasets for 'transient' Reynolds's number 180, 190 and 200 
        Because, data is not sequential at junction. Only in training data. """
    del_rows_ls = []

    if args.data_type == 'transient' or args.data_type == 'burgers1D':

        for i in range(1, n_datasets):
            idx = n_samp * i/n_datasets
            start = int(idx)
            end = int(idx) + seq_impct

            if args.operation_mode == 'offline': start = start - seq_impct            
            del_rows_ls = del_rows_ls + list(range(start, end))
        
    print(f'del_rows_ls: {del_rows_ls}')
    return np.array(del_rows_ls, dtype=np.int64)



In [17]:
"""test delStates_BW_datasets """

if 0:
    n_train = 399*3
    seq_impct = 2
    n_datasets = 3

    args.data_type = 'transient'
    delStates_BW_datasets(0, n_train, seq_impct, n_datasets)
    # del_rows_ls: [399 400 798 799]

if 0:
    n_train = 51*20
    seq_impct = 2
    n_datasets = 20

    args.data_type = 'burgers1D'
    delStates_BW_datasets(0, n_train, seq_impct, n_datasets)
    # del_rows_ls: [51, 52, 102, 103, 153,..., 919, 969, 970]

In [18]:
n_train = 399*3
seq_impct = 2
del_row = np.array([])
for e in range(int(n_train/3), int(n_train/3)+seq_impct): # if seq_impct=2 (397. 398. 399. 400.)
  del_row = np.append(del_row, [e], axis=0)    #n_train/3 = 399
for e in range(int(n_train*2/3), int(n_train*2/3)+seq_impct):  #if seq_impct=2 (796. 797. 798. 799.)
  del_row = np.append(del_row, [e], axis=0)
del_row = del_row.astype(int)
print('del_row =',del_row)
del_row-seq_impct

del_row = [399 400 798 799]


array([397, 398, 796, 797])

## ARE LSTM Net 

In [19]:
 
class are_lstm_net(nn.Module):

    def __init__(self, hidden_size, n_sensor, bottle_neck, layers, seq_len):
        super(are_lstm_net, self).__init__()
        self.hidden_size = hidden_size
        print('data_type:', args.data_type, '\n', 'using lstm cell')

        if args.data_type == 'transient': 

          self.rl = nn.LSTM(n_sensor , self.hidden_size, num_layers=layers)
          H = 200

          if args.operation_mode == 'online':      
            self.l1 = nn.Linear(hidden_size, H)
            self.l2 = nn.Linear(H, H)
            self.l3 = nn.Linear(H, bottle_neck)

          if args.operation_mode == 'offline':
            self.rlr = nn.LSTM(n_sensor, self.hidden_size, num_layers=layers)
            self.l1 = nn.Linear(hidden_size*2, H)
            self.l2 = nn.Linear(H, H)
            self.l3 = nn.Linear(H, bottle_neck)

        if args.data_type == 'periodic': 

          self.rl = nn.LSTM(n_sensor, self.hidden_size, num_layers=layers)
          H = 50
          
          if args.operation_mode == 'online':      
            self.l1 = nn.Linear(hidden_size, H)            
            self.l2 = nn.Linear(H, H)
            self.l3 = nn.Linear(H, bottle_neck)

          if args.operation_mode == 'offline':
            self.rlr = nn.LSTM(n_sensor, self.hidden_size, num_layers=layers)
            self.l1 = nn.Linear(hidden_size*2, H)
            self.l2 = nn.Linear(H, H)
            self.l3 = nn.Linear(H, bottle_neck)

        if args.data_type == 'sea_temp': 

          self.rl = nn.LSTM(n_sensor , self.hidden_size, num_layers=layers)
          H = 100

          if args.operation_mode == 'online':      
            self.l1 = nn.Linear(hidden_size, H)
            self.l2 = nn.Linear(H, H)
            self.l3 = nn.Linear(H, bottle_neck)

          if args.operation_mode == 'offline':
            self.rlr = nn.LSTM(n_sensor, self.hidden_size, num_layers=layers)
            self.l1 = nn.Linear(hidden_size*2, H)
            self.l2 = nn.Linear(H, H)
            self.l3 = nn.Linear(H, bottle_neck)

        if args.data_type == 'burgers1D': 

          self.rl = nn.LSTM(n_sensor ,self.hidden_size, num_layers=layers)
          H = 128

          if args.operation_mode == 'online':      
            self.l1 = nn.Linear(hidden_size, H)
            self.l2 = nn.Linear(H, H)
            self.l3 = nn.Linear(H, bottle_neck)


    def reset_hidden_states(self, for_batch=None):
        if for_batch is not None:
            batch_size = for_batch.shape[1]

        # Initialize recurrent hidden states
        if args.operation_mode == 'online':      
          self.rl_h = self.init_hidden(batch_size=batch_size)
          self.rl_c = self.init_hidden(batch_size=batch_size)

        if args.operation_mode == 'offline':
          self.rl_h = self.init_hidden(batch_size=batch_size)
          self.rlr_h = self.init_hidden(batch_size=batch_size)
          self.rl_c = self.init_hidden(batch_size=batch_size)
          self.rlr_c = self.init_hidden(batch_size=batch_size)

        if for_batch is not None:
            device = for_batch.device

            if args.operation_mode == 'online':      
              self.rl_h = self.rl_h.to(device)
              self.rl_c = self.rl_c.to(device)

            if args.operation_mode == 'offline':
                self.rl_h = self.rl_h.to(device)
                self.rlr_h = self.rlr_h.to(device)
                self.rl_c = self.rl_c.to(device)
                self.rlr_c = self.rlr_c.to(device)


    def init_hidden(self, batch_size):
        layers =1
        return Variable(torch.zeros(layers, batch_size, self.hidden_size))

    def forward(self, leading_dna):
        self.reset_hidden_states(for_batch=leading_dna)
        
        batch = len(leading_dna[0,:,0])
        n_sensor = len(leading_dna[0,0,:])
        seq_len = len(leading_dna[:,0,0])
        seq_impct = int((seq_len-1)/2)

        if args.operation_mode == 'online':      
          split_data1 = torch.zeros(seq_impct+1,batch,n_sensor)
          split_data1 = leading_dna[0:seq_impct+1,:,:] 

          rl_out, self.rl_h= self.rl(split_data1, (self.rl_h, self.rl_c))
          r_out = rl_out[seq_impct]

        if args.operation_mode == 'offline':
          split_data1 = torch.zeros(seq_impct+1, batch, n_sensor)
          split_data2 = torch.zeros(seq_impct+1, batch, n_sensor)
          split_data1 = leading_dna[0:seq_impct+1,:,:] #(seq_impct+1,batch,n_sensor)
          split_data2 = leading_dna[seq_impct:seq_len,:,:]

          rl_out, self.rl_h= self.rl(split_data1, (self.rl_h, self.rl_c))
          rlr_out, self.rlr_h = self.rlr(self.flip_input(split_data2), (self.rlr_h, self.rlr_c))
          r_out = torch.cat((rl_out[seq_impct], rlr_out[seq_impct]), 1)

        l1_out = F.relu(self.l1(r_out))
        l2_out = F.relu(self.l2(l1_out))
        out = l3_out = self.l3(l2_out)

        return out


    def flip_input(self, input):
        device = input.device
        flipped_array = np.flip(input.data.cpu().numpy(), 0).copy()
        return Variable(torch.from_numpy(flipped_array).to(device))


## ARE TrainLoop

In [20]:
def ARE_train_LSTM(n_sensor, seq_len, bottle_neck, hidden_size, num_epochs, lr, **kwargs):    

  print('n_sensor =',n_sensor,'seq_len = ',seq_len)
  layers = 1
  seq_impct = int((seq_len-1)/2)
  seed = kwargs.get('seed', 0)

  gtrain_out, gvalid_out, strain_in, svalid_in, _, _, gstats, stats  = ARE_train_data(n_sensor, 
                                                                                    bottle_neck,
                                                                                    seq_len, 
                                                                                    drop=kwargs.get('drop'),
                                                                                    seed=seed)

  train = sensorgcdatasetRNN(strain_in, gtrain_out)
  valid = sensorgcdatasetRNN(svalid_in, gvalid_out)
  print(f'strain_in.shape :{strain_in.shape} \ngtrain_out.shape :{gtrain_out.shape}')

  bs = len(train)  # batch size  
  bsv = len(valid)

  train_data_gen = DataLoader(train, shuffle=False, batch_size=bs)
  valid_data_gen = DataLoader(valid, shuffle=False, batch_size=bsv)

  dataloaders = {'train': train_data_gen, 'valid':valid_data_gen}
  dataset_sizes = {'train': len(train_data_gen.dataset), 'valid': len(valid_data_gen.dataset)}
  batch ={'train':bs, 'valid':bsv}

  model = are_lstm_net(hidden_size, n_sensor, bottle_neck, layers, seq_len).to(device)

  criterion = nn.MSELoss().to(device)
  optimizer = torch.optim.Adam(model.parameters(), lr=lr,  weight_decay=1e-5)

  prev_valid_loss = prev_train_loss = 100  # any large number
  stop = 0
  rerror_train = []
  rerror_valid = []
  for epoch in range(num_epochs):

    for phase in ['train']:
        running_loss=0
        if phase == 'train': model.train()
        else: model.eval()

        for i, (sensor, gc) in enumerate(dataloaders[phase]):
            # print(f'sensor_data: {sensor.shape} \n {sensor.flatten()[:7]} \n')
            # print(f'embed_data: {gc.shape} \n {gc.flatten()[:7]} \n')

            if args.operation_mode == 'online': 
                y = torch.zeros((seq_impct+1), batch[phase], n_sensor)
                for i in range(0, batch[phase]):
                    for j in range(0, (seq_impct+1)):
                        y[j, i,:] = sensor[i, n_sensor*j: n_sensor*j + n_sensor]
                    
            elif args.operation_mode == 'offline':
                y = torch.zeros(seq_len, batch[phase], n_sensor)
                for i in range(0, batch[phase]):
                    for j in range(0, seq_len):
                        y[j, i,:] = sensor[i, n_sensor*j: n_sensor*j + n_sensor]
                      
            sensor = Variable(y).to(device)
            gc = Variable(gc).to(device)

            optimizer.zero_grad()
            gc_out = model(sensor)
            loss = criterion(gc_out, gc)
            if phase=='train':
                loss.backward()
                optimizer.step()

            running_loss = gc_out.shape[0] * loss.data + running_loss

        if phase == 'train': train_epoch_loss = running_loss/dataset_sizes[phase]
        elif phase == 'valid': valid_epoch_loss = running_loss/dataset_sizes[phase]
    stop = stop + 1

    progress = f'   ({epoch}) Training loss: {train_epoch_loss:.8f}'
    if epoch%50 == 0: print(progress)

    if (train_epoch_loss < prev_train_loss):
        model_wts = model.state_dict()
        prev_train_loss = train_epoch_loss
        stop = 0
        output_last = gc_out
        gc_last = gc
    
    if stop == 230:
        print('Early stopping criteria fulfilled')
        break

    rerror_train.append(train_epoch_loss)

  path_to_weights = get_path('are', 'W', seed=seed)
  weights_name = get_name('are', 'W', drop=kwargs.get('drop'), n_sensor=n_sensor, seq_len=seq_len, bottle_neck=bottle_neck)
  torch.save(model_wts, osp.join(path_to_weights, weights_name))
  print('=========saved weights=========')

  Plots().plot_graph(rerror_train, rerror_train, savePath=osp.join(path_to_weights, weights_name))

## ARE TestLoop

In [21]:
def ARE_test(n_sensor, seq_len, bottle_neck, plot, save, hidden_size = 50, **kwargs):
    """
    ARGS:
        plot (tuple): 0 or 1 ()
        save (tuple): 0 or 1 ()

    VARS:
        states_pred (torch.tensor): [n_test_samp, n_pixel] 
        stats (torch.tensor): [2, n_pixel]

    RETURNS: average L2 error accros all test samples
    """
  
    layers = 1
    seq_impct = int((seq_len-1)/2)
    SNR = kwargs.get('SNR', None)
    seed = kwargs.get('seed', 0)

    if args.data_type == 'periodic': start, end = 0, 60
    if args.data_type == 'transient': start, end = 200, 399
    if args.data_type == 'turbulence': start, end = 0, 400
    if args.data_type == 'sea_temp': start, end = 0, 200 
    if args.data_type == 'burgers1D': start, end = 0, 51

    # ------------------------------ Load data -----------------------------------
    _, _, _, _, embed_data, sensor_data, embed_stats, stats = ARE_train_data(n_sensor, bottle_neck, seq_len, 
                                                                    drop=kwargs.get('drop',None), 
                                                                    SNR=SNR, plot_SNR=kwargs.get('plot_SNR'),
                                                                    plot=plot[2], save=save[2],
                                                                    seed=seed)
    """ 
    sensor_data (np.array): [n_test_samp, (seq_impct+1)*n_sensor]
    embed_stats (np.array): [n_test_samp, n_embed]
    """
    datasetClass = sensorgcdatasetRNN(sensor_data, embed_data)
    testLoader = DataLoader(datasetClass, shuffle=False, batch_size=1)

    # -------------------------- load model weights ------------------------------
    model = are_lstm_net(hidden_size, n_sensor, bottle_neck, layers, seq_len).to(device)

    weightsDir = get_path('are', 'W', seed=seed)
    weightsName = get_name('are', 'W', drop=kwargs.get('drop'), n_sensor=n_sensor, 
                            seq_len=seq_len, bottle_neck=bottle_neck)
    # weightsName = get_name('are', 'W', n_sensor=n_sensor, 
    #                         seq_len=7, bottle_neck=bottle_neck)
    pretrained_weightsRNN = osp.join(weightsDir, weightsName)
    print(f'LSTM weights path: {pretrained_weightsRNN}')

    model.load_state_dict(torch.load(pretrained_weightsRNN, map_location=torch.device(device)))
    model.eval()

    # -------------------------- predict embeddings ------------------------------
    gc_pred = np.zeros([len(datasetClass), bottle_neck])
    for i, (sensor, gc) in enumerate(testLoader):
        """ sensor: [1, ]"""
        """ 
        sensor (torch.tensor): [1, (seq_impct+1)*n_sensor]
        gc (torch.tensor): [1, n_embed]
        """

        if args.operation_mode == 'online': 
                y = torch.zeros((seq_impct+1), 1, n_sensor)
                for j in range(0, (seq_impct+1)):
                    y[j, 0, :] = sensor[0, n_sensor*j: (n_sensor*j + n_sensor)]
                    
        elif args.operation_mode == 'offline':
                y = torch.zeros(seq_len, 1, n_sensor)
                for j in range(0, seq_len):
                    y[j, 0, :] = sensor[0, n_sensor*j: (n_sensor*j + n_sensor)]
                
        gc_pred[i, :] = model(y.to(device)).cpu().data.numpy()
        
    gc_pred = gc_pred * embed_stats[1, :] + embed_stats[0, :]

    # ----------------------------- Load embeddings ------------------------------
    _, _, testAE, statsAE = process_data()
    n_test = testAE.shape[0]
    end = n_test
    del_TestRows_ls = delStates_BW_datasets(n_test, seq_impct, args.n_TestDatasets)
    target = np.delete(testAE, del_TestRows_ls, 0)

    if args.operation_mode == 'online': 
        AEdatasetClass = sensorgcdatasetAA(gc_pred, target[seq_impct:n_test, :], statsAE)
    elif args.operation_mode == 'offline':
        AEdatasetClass = sensorgcdatasetAA(gc_pred, target[seq_impct:n_test-seq_impct, :], statsAE)
    
    AEtestLoader = DataLoader(AEdatasetClass, shuffle=False, batch_size=1)
    _, cords = sensor_cord_data(n_sensor, seed=kwargs.get('seed', 0))

    # ------------------------- Load autoencoder weights -------------------------
    path_to_weights = get_path('are', 'W_AE')
    weights_name = get_name('are', 'W_AE', drop=kwargs.get('drop'), bottle_neck=bottle_neck)
    pretrained_weightsAE = osp.join(path_to_weights, weights_name)
    print(f'Autoencoder weights path: {pretrained_weightsAE}')

    model = autoencoder(bottle_neck, drop=kwargs.get('drop')).to(device)

    model.load_state_dict(torch.load(pretrained_weightsAE, map_location=torch.device(device)))
    model.eval()

    # ---------------------------- predict states --------------------------------
    t_pred = np.zeros([len(AEtestLoader), len(testAE[0,:]) ])
    t_true = np.zeros([len(AEtestLoader), len(testAE[0,:]) ])
    for i, (gc_pre, t) in enumerate(AEtestLoader):
        t_out_tensor = model(gc_pre.to(device))
        t_out = t_out_tensor.cpu().data.numpy()
        t_pred[i, :] = t_out
        t_true[i, :] = t

    if kwargs.get('use_stats', 1):
            # t_pred = t_pred * statsAE[1, :] + statsAE[0, :]
            t_pred = t_pred * stats[1, :] + stats[0, :]
            t_true = t_true * statsAE[1, :] + statsAE[0, :]

    # -------------------------- Calc average error ------------------------------
    Ferr = []
    if args.operation_mode == 'online': end = end-seq_impct*args.n_TestDatasets
    elif args.operation_mode == 'offline': end = end-2*seq_impct*args.n_TestDatasets

    # pdb.set_trace()
    for i in range(start,end):
        Ferr.append(np.linalg.norm(t_true[i,:] - t_pred[i,:]) / np.linalg.norm(t_true[i,:]))
    
    errorAvg = np.mean(Ferr)
    print(f'Error: {errorAvg}')

  
    # ------------------------- Save pred for plotting ---------------------------
    state_pred = np.zeros([len(testAE), len(testAE[0,:]) ])
    statesPerDataset = len(testAE)//args.n_TestDatasets
    statesPredPerDataset = len(AEtestLoader)//args.n_TestDatasets
    # pdb.set_trace()
    for i in range(args.n_TestDatasets):
        state_pred[i*statesPerDataset+seq_impct: (i+1)*statesPerDataset] = \
        t_pred[i*statesPredPerDataset: (i+1)*statesPredPerDataset]
    
    predData_Path = osp.join(weightsDir, f'predData_s{n_sensor}_seqlen{seq_len}_SNR{SNR}.hdf5')
    with h5py.File(predData_Path, 'w') as f:
        f.create_dataset('pred', data=state_pred)
        f.create_dataset('true', data=t_true)
        f.create_dataset('coords', data=cords)
        f.create_dataset('method', data=np.array(list('ARE'.encode('utf8'))))

    print(f'pred data saved at {predData_Path}')

    return float(errorAvg)

# POD DEEP STATE (PDS)

## PDS_train_data

In [22]:
class sensorgcdatasetDS(Dataset):

    def __init__(self, in_file, out_file, transform=None):
        self.in_frame = in_file
        self.out_frame = out_file
        self.transform = transform

    def __len__(self):
        return len(self.in_frame)

    def __getitem__(self, idx):
        inn = self.in_frame[idx, :]
        out = self.out_frame[idx, :]
        inn = torch.from_numpy(inn).float()
        out = torch.from_numpy(out).float()
        return inn, out

def PDS_train_data(sensor_num, bottle_neck, **kwargs):
  
  trainAE, validAE, testAE, stats = process_data(SNR=kwargs.get('SNR'), plot=kwargs.get('plot'), save=kwargs.get('save'),
                                                 tr_samp=kwargs.get('tr_samp'), plot_SNR=kwargs.get('plot_SNR'),
                                                 seed=kwargs.get('seed', 0))
  mean, std = stats[0], stats[1]

  # ----------------------------------------------------------------------------
  X = (trainAE - mean).transpose()
  phi, s, V = np.linalg.svd(X, full_matrices=False)
  del(s,V)
  phi = phi[:,0:bottle_neck]

  gtrain_out = (np.dot(phi.transpose(), X)).transpose()
  del(X)

  XX = (validAE - mean).transpose()
  gvalid_out = (np.dot(phi.transpose(), XX)).transpose()

  XXX = (testAE - mean).transpose()
  gtest_out = (np.dot(phi.transpose(), XXX)).transpose()
  del(XX, XXX)
  # ----------------------------------------------------------------------------

  trainAE_stan, validAE_stan, testAE_stan = standardize(trainAE, validAE, testAE, stats)
  del(trainAE, validAE, testAE)

  strain_in, svalid_in, stest_in = sensor_data(trainAE_stan, validAE_stan, testAE_stan, sensor_num)

  gstats = np.array([np.mean(gtrain_out, axis = 0), np.std(gtrain_out, axis = 0)])
  gtrain_out, gvalid_out, gtest_out = standardize(gtrain_out, gvalid_out, gtest_out, gstats)

  del(trainAE_stan,validAE_stan,testAE_stan)

  return gtrain_out, gvalid_out, strain_in, svalid_in, gtest_out, stest_in, gstats, phi, stats   

## PDS net

In [23]:
class PDSnetwork(nn.Module):

    def __init__(self, n_sensor, bottle_neck):
        super(PDSnetwork, self).__init__()

        if args.data_type == 'transient':
            H = 200
        if args.data_type == 'periodic':
            H = 50
        if args.data_type == 'sea_temp':
            H = 100
        if args.data_type == 'burgers1D': 
            H = 128

        self.l1 = nn.Linear(n_sensor, H)
        self.l2 = nn.Linear(H, H)
        self.output = nn.Linear(H, bottle_neck)

    def forward(self, leading_dna):
        l1_out = F.relu(self.l1(leading_dna))
        l2_out = F.relu(self.l2(l1_out))
        output = self.output(l2_out)        
        return output

## PDS TrainLoop

In [24]:
def PDS_train(n_sensor, bottle_neck, num_epochs, lr, **kwargs):
  
  gtrain_out, gvalid_out, strain_in, svalid_in, gtest_stan, stest_DS, gstats, phi, stats = PDS_train_data(n_sensor, bottle_neck,
                                                                                                          seed=kwargs.get('seed', 0))


  del(gtest_stan, stest_DS,phi, stats)

  train = sensorgcdatasetDS(strain_in, gtrain_out)
  valid = sensorgcdatasetDS(svalid_in, gvalid_out)

  bs = len(train)  # batch size
  bsv = len(valid)
  train_data_gen = DataLoader(train, shuffle=True, batch_size=bs)
  valid_data_gen = DataLoader(valid, shuffle=True, batch_size=bsv)

  dataloaders = {'train': train_data_gen, 'valid':valid_data_gen}
  dataset_sizes = {'train': len(train_data_gen.dataset), 'valid': len(valid_data_gen.dataset)}
  batch ={'train':bs, 'valid':bsv}

  model = PDSnetwork(n_sensor, bottle_neck).to(device)

  # Loss and Optimizer
  criterion = nn.MSELoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)
  #exp_lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[500], gamma=0.1)

  prev_valid_loss = prev_train_loss = 100  # any large number
  stop = 0
  rerror_train = []
  rerror_valid = []
  for epoch in range(num_epochs):
    for phase in ['train','valid']:
      running_loss=0
      if phase == 'train':
        model.train()
      else:
        model.eval()

      for i, (sensor, gc) in enumerate(dataloaders[phase]):
          sensor = Variable(sensor).to(device)
          gc = Variable(gc).to(device)
          optimizer.zero_grad()
          gc_out = model(sensor)

          # calculate loss
          loss = criterion(gc_out, gc)

          if phase=='train':
            loss.backward()
            optimizer.step()
            #exp_lr_scheduler.step()

          running_loss = gc_out.shape[0] * loss.data + running_loss

      if phase == 'train':
        train_epoch_loss = running_loss/dataset_sizes[phase]
      elif phase == 'valid':
        valid_epoch_loss = running_loss/dataset_sizes[phase]
    stop = stop + 1

    progress = f'   ({epoch}) Training loss: {train_epoch_loss:.8f}'
    if epoch%50 == 0: print(progress)
    
    if (train_epoch_loss < prev_train_loss):
      model_wts = model.state_dict()
      prev_valid_loss = valid_epoch_loss
      prev_train_loss = train_epoch_loss
      stop = 0

    if stop == 230:
        print('Early stopping criteria fulfilled')
        break
    rerror_train.append(train_epoch_loss)
    rerror_valid.append(valid_epoch_loss)

  path_to_weights = get_path('pds', 'W')
  weights_name = get_name('pds', 'W', drop=kwargs.get('drop'), n_sensor=n_sensor, bottle_neck=bottle_neck)
  torch.save(model_wts, osp.join(path_to_weights, weights_name))
  print('=========saved weights=========')

  Plots().plot_graph(rerror_train, rerror_valid, savePath=osp.join(path_to_weights, weights_name))

## PDS TestLoop

In [25]:

def PDS_test(n_sensor, bottle_neck, plot, save, **kwargs): 
    SNR=kwargs.get('SNR',None)

    path_to_weights = get_path('pds', 'W')
    weights_name = get_name('pds', 'W', n_sensor=n_sensor, bottle_neck=bottle_neck)
    pretrained_weightsRNN = osp.join(path_to_weights, weights_name)
    
    gtrain_out, gvalid_out, strain_in, svalid_in, gtest, stest_in, gstats, phi, stats = PDS_train_data(n_sensor,bottle_neck,
                                                                                                        SNR=SNR,
                                                                                                        plot_SNR=kwargs.get('plot_SNR'),
                                                                                                        plot=plot[2], save=save[2],
                                                                                                        seed=kwargs.get('seed', 0))

    del(gtrain_out, gvalid_out, strain_in, svalid_in)

    modell = PDSnetwork(n_sensor, bottle_neck).to(device)
    modell.load_state_dict(torch.load(pretrained_weightsRNN, map_location=torch.device(device)))
    modell.eval()

    test = sensorgcdatasetDS(stest_in, gtest)

    gc_pred = np.zeros([len(test), bottle_neck])
    gc_pod = np.zeros([len(test), bottle_neck])
    for i, (sensor, gc) in enumerate(test):
        y = modell(sensor.to(device))
        gc_pred[i, :] = y.cpu().data.numpy()
        gc_pod[i, :] = gc.cpu().data.numpy()

    gc_pred = gc_pred * gstats[1, :] + gstats[0, :]
    gc_pod = gc_pod * gstats[1, :] + gstats[0, :]

    t_pred = gc_pred.dot(phi.T) 
    t_pod = gc_pod.dot(phi.T) 
    
    _, _, testAE, statsAE = process_data()
    n_test = testAE.shape[0]
    end = n_test
    t_true = testAE 

    if kwargs.get('use_stats', 1):
        # t_pred = t_pred + statsAE[0]
        t_pred = t_pred + stats[0]
    else:
        t_true = t_true - statsAE[0]

    _, cords = sensor_cord_data(n_sensor, seed=kwargs.get('seed', 0))

    # ----------------------------------------------------------------------------
    
    Ferr = []
    if args.data_type == 'periodic': start, end = 0, 60
    if args.data_type == 'transient': start, end = 200, 399
    if args.data_type == 'turbulence': start, end = 0, 400
    if args.data_type == 'sea_temp': start, end = 0, 200 
    if args.data_type == 'burgers1D': start, end = 0, n_test

    # if args.operation_mode == 'online': end = end-seq_impct
    # elif args.operation_mode == 'offline': end = end-2*seq_impct

    for i in range(start,end):
        Ferr.append(np.linalg.norm(t_true[i,:] - t_pred[i,:]) / np.linalg.norm(t_true[i,:]))

    Ferr_avg = np.mean(Ferr)
    print("Error: ", Ferr_avg, '------------------------------------------------')

    predData_Path = osp.join(path_to_weights, f'predData_s{n_sensor}_SNR{SNR}.hdf5')
    with h5py.File(predData_Path, 'w') as f:
        f.create_dataset('pred', data=t_pred)
        f.create_dataset('true', data=t_true)
        f.create_dataset('coords', data=cords)
        f.create_dataset('method', data=np.array(list('PDS'.encode('utf8'))))
    
    return float(Ferr_avg)

# SHALLOW DECODER (SD)

## SD_train_data

In [26]:
def SD_train_data(sensor_num, **kwargs):

  trainAE, validAE, testAE, stats = process_data(SNR=kwargs.get('SNR'), plot=kwargs.get('plot'), save=kwargs.get('save'),
                                                tr_samp=kwargs.get('tr_samp'), plot_SNR=kwargs.get('plot_SNR'),
                                                seed=kwargs.get('seed', 0))
  stats[1] = np.ones_like(stats[1])

  trainAE_stan, validAE_stan, testAE_stan = standardize(trainAE, validAE, testAE, stats)

  strain_in, svalid_in, stest_in = sensor_data(trainAE_stan, validAE_stan, testAE_stan, sensor_num)

  return trainAE_stan, validAE_stan, strain_in, svalid_in, testAE_stan, stest_in, stats

## Shallow Decoder Net

In [27]:
class shallow_decoder_net(nn.Module):
    def __init__(self, n_sensors, **kwargs):
        super(shallow_decoder_net, self).__init__()
        
        m, n = img_dims()
        self.n_sensors = n_sensors
        self.outputlayer_size = m*n
        self.drop = kwargs.get('drop')

        if self.drop: print('shallow_decoder oprating with drop')
        else: print('shallow_decoder oprating without drop')

        if args.data_type == 'periodic':
            N1, N2 = 35, 40
        if args.data_type == 'transient':
            N1, N2 = 350, 400
        if args.data_type == 'turbulence':
            N1, N2 = 350, 400
        if args.data_type == 'sea_temp':
            N1, N2 = 350, 400
            self.outputlayer_size = 44219
        if args.data_type == 'burgers1D': 
            N1, N2 = 128, 256
        
        if args.exp == 'SDnets':
            SDnet = kwargs.get('SDnet', '')
            N1, N2 = SDnet[0], SDnet[1] 
            print(f'N1 {N1}, N2 {N2}')
        
        self.learn_features = nn.Sequential(         
            nn.Linear(n_sensors, N1),
            nn.ReLU(True), 
            nn.BatchNorm1d(1),  
            )        
        
        self.learn_coef = nn.Sequential(            
            nn.Linear(N1, N2),
            nn.ReLU(True),  
            nn.BatchNorm1d(1),  
            )

        self.learn_dictionary = nn.Sequential(
            nn.Linear(N2, self.outputlayer_size),
            )
        

        for m in self.modules():
            # torch.manual_seed(args.seed)
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)        

            elif isinstance(m, nn.Linear):
                nn.init.xavier_normal(m.weight)
                if m.bias is not None:
                    nn.init.constant(m.bias, 0.0)    


    def forward(self, x):
        """
        ARGS:
            x (torch.tensor): [n_samp, 1, n_sensors]
        """
        # print(f'x: {x.shape}')
        x = self.learn_features(x)
        if self.drop:
            # torch.manual_seed(args.seed)
            x = nn.functional.dropout(x, p=0.1, training=self.training)   
        x = self.learn_coef(x)
        x = self.learn_dictionary(x) 
        return x

## SD Train Loop

In [28]:
def SD_train(n_sensor, num_epochs, lr, wd, learning_rate_change=0.9, weight_decay_change=0.8, epoch_update=100, **kwargs):    

  print('n_sensor =',n_sensor)
  train_out, valid_out, strain_in, svalid_in, _, _, stats = SD_train_data(n_sensor, seed=kwargs.get('seed', 0))

  strain_in, svalid_in  = torch.tensor(np.expand_dims(strain_in, axis=1)).float().to(device), torch.tensor(np.expand_dims(svalid_in, axis=1)).float().to(device)
  train_out, valid_out = torch.tensor(np.expand_dims(train_out, axis=1)).float().to(device), torch.tensor(np.expand_dims(valid_out, axis=1)).float().to(device)

  train_data = torch.utils.data.TensorDataset(strain_in, train_out)
  valid_data = torch.utils.data.TensorDataset(svalid_in, valid_out)

  bs = round(len(strain_in)/3)  # batch size
  bsv = len(svalid_in)

  train_loader = DataLoader(dataset = train_data, batch_size = bs, shuffle = True)
  valid_loader = DataLoader(dataset = valid_data, batch_size = bsv, shuffle = True)

  dataloaders = {'train': train_loader, 'valid':valid_loader}
  dataset_sizes = {'train': len(train_loader.dataset), 'valid': len(valid_loader.dataset)}
  batch ={'train':bs, 'valid':bsv}

  model = shallow_decoder_net(n_sensor, drop=kwargs.get('drop',None), SDnet=kwargs.get('SDnet')).to(device)

  criterion = nn.MSELoss().to(device)
  optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=wd)

  def exp_lr_scheduler(optimizer, epoch, lr_decay_rate=0.8, weight_decay_rate=0.8, lr_decay_epoch=100):
        """Decay learning rate by a factor of lr_decay_rate every lr_decay_epoch epochs"""
        if epoch % lr_decay_epoch:
            return 
        
        # if args.optimizer == 'sgd':
        for param_group in optimizer.param_groups:
            param_group['lr'] *= lr_decay_rate
            param_group['weight_decay'] *= weight_decay_rate
        return 

  prev_valid_loss = prev_train_loss = 100  # any large number
  stop = 0
  rerror_train = []
  rerror_valid = []

  for epochs in range(num_epochs):
    for phase in ['train','valid']:
      running_loss=0
      if phase == 'train':
        model.train()
      else:
        model.eval()

      for i, (sensor, true) in enumerate(dataloaders[phase]):

          optimizer.zero_grad()
          pred = model(sensor).to(device)
          loss = criterion(pred, true)

          if phase=='train':
            loss.backward()
            optimizer.step()
            # ===================adjusted lr========================
            exp_lr_scheduler(optimizer, epochs, lr_decay_rate = learning_rate_change, 
                             weight_decay_rate = weight_decay_change, 
                             lr_decay_epoch = epoch_update)

          running_loss = pred.shape[0] * loss.data + running_loss

      if phase == 'train':
        train_epoch_loss = running_loss/dataset_sizes[phase]
        train_out_last = pred 
        train_in_last = true 
      elif phase == 'valid':
        valid_epoch_loss = running_loss/dataset_sizes[phase]
    stop = stop + 1

    progress = f'   ({epochs}) Training loss: {train_epoch_loss:.8f}'
    if epochs%50 == 0: print(progress)
    
    # if (valid_epoch_loss < prev_valid_loss):
    if (train_epoch_loss < prev_train_loss):
      model_wts = model.state_dict()
      prev_valid_loss = valid_epoch_loss
      prev_train_loss = train_epoch_loss

      stop = 0
      best_train_out, best_train_in = train_out_last, train_in_last
      best_valid_out, best_valid_in = pred, true

    
    if stop == 230:
        print('Early stopping criteria fulfilled')
        break

    rerror_train.append(train_epoch_loss)
    rerror_valid.append(valid_epoch_loss)

  print('({}) BEST_Training Loss: {:.8f} BEST_Valid Loss: {:.8f} '.format(epochs, prev_train_loss, prev_valid_loss))

  path_to_weights = get_path('sd', 'W', **kwargs)
  weights_name = get_name('sd', 'W', drop=kwargs.get('drop'), n_sensor=n_sensor)
  torch.save(model_wts, osp.join(path_to_weights, weights_name))
  print('=========saved weights=========')

  Plots().plot_graph(rerror_train,rerror_valid, osp.join(path_to_weights, weights_name))
  print('================================ over ================================')

## SD TestLoop

In [29]:
def SD_test(n_sensor, plot, save, **kwargs):

    SNR=kwargs.get('SNR',None)

    path_to_weights = get_path('sd', 'W', **kwargs)
    weights_name = get_name('sd', 'W', drop=kwargs.get('drop'), n_sensor=n_sensor)
    pretrained_weights = osp.join(path_to_weights, weights_name)

    train_out, valid_out, strain_in, svalid_in, test_out, stest_in, stats = SD_train_data(n_sensor, SNR=SNR,
                                                                                            plot_SNR=kwargs.get('plot_SNR'),
                                                                                            plot=plot[2], save=save[2],
                                                                                            seed=kwargs.get('seed', 0))
    del(train_out, valid_out, strain_in, svalid_in)
    test_out, stest_in  = torch.tensor(np.expand_dims(test_out, axis=1)).float().to(device), torch.tensor(np.expand_dims(stest_in, axis=1)).float().to(device)
    
    model = shallow_decoder_net(n_sensor, drop=kwargs.get('drop'), SDnet=kwargs.get('SDnet')).to(device)
    print(model)

    # Load saved neural network weights
    model.load_state_dict(torch.load(pretrained_weights, map_location=torch.device(device)))
    model.eval()

    test = torch.utils.data.TensorDataset(stest_in, test_out)
    test_data_genn = DataLoader(test, shuffle=False, batch_size=1)

    _, cords = sensor_cord_data(n_sensor, seed=kwargs.get('seed', 0))      

    states_pred = np.zeros([len(test), len(test_out[0, 0, :]) ])
    states_true = np.zeros([len(test), len(test_out[0, 0, :]) ])

    for i, (sensor, state) in enumerate(test):
        pred = (model(sensor[:, None].to(device)))[:, 0]
        states_pred[i, :] = pred.cpu().data.numpy()
        states_true[i, :] = state.cpu().data.numpy()

    assert (states_true == test_out[:, 0].cpu().data.numpy()).any()

    trainAE, validAE, testAE, statsAE = process_data()
    del(trainAE,validAE)
    n_test = testAE.shape[0]
    statsAE[1] = np.ones_like(statsAE[1])

    states_true = testAE

    if kwargs.get('use_stats', 1):
        # states_pred = states_pred + statsAE[0, :]
        states_pred = states_pred + stats[0, :]
    else:
        states_true = (testAE - statsAE[0, :])  # / statsAE[1, :]

    Ferr = []
    if args.data_type == 'periodic': start, end = 0, 60
    if args.data_type == 'transient': start, end = 200, 399
    if args.data_type == 'turbulence': start, end = 0, 400
    if args.data_type == 'sea_temp': start, end = 0, 200 
    if args.data_type == 'burgers1D': start, end = 0, n_test

    # if args.operation_mode == 'online': end = end-seq_impct
    # elif args.operation_mode == 'offline': end = end-2*seq_impct
    
    for i in range(start, end):
        Ferr.append(np.linalg.norm(states_true[i,:] - states_pred[i,:]) / np.linalg.norm(states_true[i,:]))

    Ferr_avg = np.mean(Ferr)
    print("Error: ", Ferr_avg, '------------------------------------------------')

    predData_Path = osp.join(path_to_weights, f'predData_s{n_sensor}_SNR{SNR}.hdf5')
    with h5py.File(predData_Path, 'w') as f:
        f.create_dataset('pred', data=states_pred)
        f.create_dataset('true', data=states_true)
        f.create_dataset('coords', data=cords)
        f.create_dataset('method', data=np.array(list('SD'.encode('utf8'))))
    
    return float(Ferr_avg)

# Final results

In [30]:
def append_graph(graphData):
    error_path = osp.join(get_path('', 'P'), 'error.yaml')
    if not os.path.exists(error_path): 
        errorPlots = graphData
        yaml.dump(errorPlots, open(error_path, 'w'))
    else: 
        errorPlots_ = yaml.load(open(error_path, 'r'))
        errorPlots = {**errorPlots_, **graphData}
        yaml.dump(errorPlots, open(error_path, 'w'))

## Periodic

### method and sensors

In [ ]:
def periodic_mthd__snsr():

  num_epochs, lr, bs, bsv, bn, early_stop  = 1200, 0.0007, 399, 133, 25, 200
  AutoEcoder(bn, lr, num_epochs, bs, bsv, early_stop, drop=0)


  for s in [1, 2, 5, 10]:

      num_epochs, lr, hidden_size, seq_len, bn  = 2700, 0.0006, 50, 7, 25
      ARE_train_LSTM(s, seq_len, bn, hidden_size, num_epochs, lr, drop=0)

      num_epochs, lr, bn  = 1200, 0.0009, 25
      PDS_train(s, bn, num_epochs, lr)

      num_epochs, lr, wd  = 1500, 0.001, 1e-4
      SD_train(s, num_epochs, lr, wd)

  error = np.zeros((3, 4))
  plot_s_list = np.array([1, 2])
  plot_seq_len_list = np.array([7])
  plot_image_idx = np.array([39])  

  for s, i in zip([1], count(0, 1)):  #[1, 2, 5, 10]
    seq_len, bn, plot, save = 7, 25, (1, 1, 0), (1, 1, 0)

    er1 = ARE_test(s, seq_len, bn,
                  plot, save,
                  plot_s_list=plot_s_list, 
                  plot_seq_len_list=plot_seq_len_list,
                  plot_image_idx=plot_image_idx)
    
    er2 = PDS_test(s, bn, plot, save,
                   plot_s_list=plot_s_list,
                   plot_image_idx=plot_image_idx)

    er3 = SD_test(s, plot, save,
                  plot_s_list=plot_s_list,
                  plot_image_idx=plot_image_idx)

    error[:, i] = [er1, er2, er3]

  error_path = get_path('', 'P')
  np.savetxt(osp.join(error_path, "error_{}.csv".format(args.RNN)), error, delimiter=",")


def imgPlot_fig9():
    n_sensor = 1
    seq_len = 7
    SNR = None
    try:
        predData_Path = osp.join(get_path('are', 'W'), f'predData_s{n_sensor}_seqlen{seq_len}_SNR{SNR}.hdf5')
        ARE_predData = h5py.File(predData_Path, 'r')
        print(f'loaded ARE pred data')

        predData_Path = osp.join(get_path('pds', 'W'), f'predData_s{n_sensor}_SNR{SNR}.hdf5')
        PDS_predData = h5py.File(predData_Path, 'r')
        print(f'loaded PDS pred data')

        predData_Path = osp.join(get_path('sd', 'W'), f'predData_s{n_sensor}_SNR{SNR}.hdf5')
        SD_predData = h5py.File(predData_Path, 'r')
        print(f'loaded SD pred data')
    except:
        raise Exception(FileNotFoundError)

    plotParams = 39  # index of image to be plotted as data contains many predictions
    saveplot_Path = osp.join(get_path('are', 'P'), f'imgPlot_fig9')
    Plots().periodicFlow_fig(1, saveplot_Path, (ARE_predData, PDS_predData, SD_predData, plotParams))



args.data_type = 'periodic' 
args.exp = 'mthd__snsr'

periodic_mthd__snsr()
imgPlot_fig9()

### method and SNR

In [ ]:
from posixpath import join
def periodic_mthd__snr():
  
  num_epochs, lr, bs, bsv, bn, early_stop = 1200, 0.0007, 399, 133, 25, 200
  AutoEcoder(bn, lr, num_epochs, bs, bsv, early_stop, drop=0.35)

  for s in [1, 2]:

      num_epochs, lr, hidden_size, seq_len, bn  = 2700, 0.0006, 50, 7, 25
      ARE_train_LSTM(s, seq_len, bn, hidden_size, num_epochs, lr, drop=0.35)

      num_epochs, lr, bn  = 1200, 0.0009, 25
      PDS_train(s, bn, num_epochs, lr)

      num_epochs, lr, wd  = 1500, 0.001, 1e-4
      SD_train(s, num_epochs, lr, wd, drop=0.1)

  SNR_range = [20]  # range(16, 84, 4)
  error = np.zeros((3, len(SNR_range)))
  plot_s_list = np.array([1, 2])
  plot_seq_len_list = np.array([7])
  plot_image_idx = np.array([39])
  plot_SNR = np.array([20, 72])
  
  for s in [1]: # [1, 2]
    for SNR, i in zip(SNR_range, count(0, 1)):
      seq_len, bn, plot, save, use_stats = 7, 25, (1, 1, 1), (1, 1, 1), 1
      print('\n no. of sensors:',s,' SNR:',SNR)

      er1 = ARE_test(s, seq_len, bn,
                    plot, save,
                    drop=0.35, SNR=SNR,
                    plot_s_list=plot_s_list, 
                    plot_seq_len_list=plot_seq_len_list,
                    plot_image_idx=plot_image_idx,
                    plot_SNR=plot_SNR,
                    use_stats=use_stats)
      
      er2 = PDS_test(s, bn, plot, save,
                    drop=1, SNR=SNR,
                    plot_s_list=plot_s_list,
                    plot_image_idx=plot_image_idx,
                    plot_SNR=plot_SNR,
                    use_stats=use_stats)

      er3 = SD_test(s, plot, save,
                    drop=0.1, SNR=SNR,
                    plot_s_list=plot_s_list,
                    plot_image_idx=plot_image_idx,
                    plot_SNR=plot_SNR,
                    use_stats=use_stats)

      error[:, i] = [er1, er2, er3]

    error_path = get_path('', 'P')
    np.savetxt(osp.join(error_path, "error_{}_{}.csv".format(args.RNN, s)), error, delimiter=",")


def imgPlot_fig10():

    n_sensor = 1
    seq_len = 7
    SNR = 20
    try:
        predData_Path = osp.join(get_path('are', 'W'), f'predData_s{n_sensor}_seqlen{seq_len}_SNR{SNR}.hdf5')
        ARE_predData = h5py.File(predData_Path, 'r')
        print(f'loaded ARE pred data')

        predData_Path = osp.join(get_path('pds', 'W'), f'predData_s{n_sensor}_SNR{SNR}.hdf5')
        PDS_predData = h5py.File(predData_Path, 'r')
        print(f'loaded PDS pred data')

        predData_Path = osp.join(get_path('sd', 'W'), f'predData_s{n_sensor}_SNR{SNR}.hdf5')
        SD_predData = h5py.File(predData_Path, 'r')
        print(f'loaded SD pred data')
    except:
        raise Exception(FileNotFoundError)

    plotParams = 39  # index of image to be plotted as data contains many predictions
    saveplot_Path = osp.join(get_path('are', 'P'), f'imgPlot_fig10')
    Plots().periodicFlow_fig(1, saveplot_Path, (ARE_predData, PDS_predData, SD_predData, plotParams))


args.data_type = 'periodic' 
args.exp = 'mthd__snr'

periodic_mthd__snr()
imgPlot_fig10()

## Transient

### method and sensors

In [ ]:
def transient_mthd__snsr():

  num_epochs, lr, bs, bsv, bn, early_stop  = 1200, 0.001, 399, 133, 25, 200
  AutoEcoder(bn, lr, num_epochs, bs, bsv, early_stop, drop=0)


  for s in [1, 2, 5, 10]:

      num_epochs, lr, hidden_size, seq_len, bn  = 2600, 0.0008, 50, 7, 25
      ARE_train_LSTM(s, seq_len, bn, hidden_size, num_epochs, lr, drop=0)

      num_epochs, lr, bn  = 1500, 0.0009, 25
      PDS_train(s, bn, num_epochs, lr)

      num_epochs, lr, wd  = 1500, 0.001, 1e-4
      SD_train(s, num_epochs, lr, wd)

  error = np.zeros((3, 4))
  plot_s_list = np.array([1, 2])
  plot_seq_len_list = np.array([7])
  plot_image_idx = np.array([335])    

  for s, i in zip([1], count(0, 1)):  # [1, 2, 5, 10]
    seq_len, bn, plot, save = 7, 25, (1, 1, 0), (1, 1, 0)

    er1 = ARE_test(s, seq_len, bn,
                  plot, save,
                  plot_s_list=plot_s_list, 
                  plot_seq_len_list=plot_seq_len_list,
                  plot_image_idx=plot_image_idx)
    
    er2 = PDS_test(s, bn, plot, save,
                   plot_s_list=plot_s_list,
                   plot_image_idx=plot_image_idx)

    er3 = SD_test(s, plot, save,
                  plot_s_list=plot_s_list,
                  plot_image_idx=plot_image_idx)

    error[:, i] = [er1, er2, er3]

  error_path = get_path('', 'P')
  np.savetxt(osp.join(error_path, "error_{}.csv".format(args.RNN)), error, delimiter=",")


def imgPlot_fig12():

    n_sensor = 1
    seq_len = 7
    SNR = None
    try:
        predData_Path = osp.join(get_path('are', 'W'), f'predData_s{n_sensor}_seqlen{seq_len}_SNR{SNR}.hdf5')
        ARE_predData = h5py.File(predData_Path, 'r')
        print(f'loaded ARE pred data')

        predData_Path = osp.join(get_path('pds', 'W'), f'predData_s{n_sensor}_SNR{SNR}.hdf5')
        PDS_predData = h5py.File(predData_Path, 'r')
        print(f'loaded PDS pred data')

        predData_Path = osp.join(get_path('sd', 'W'), f'predData_s{n_sensor}_SNR{SNR}.hdf5')
        SD_predData = h5py.File(predData_Path, 'r')
        print(f'loaded SD pred data')
    except:
        raise Exception(FileNotFoundError)

    plotParams = 335  # index of image to be plotted as data contains many predictions
    saveplot_Path = osp.join(get_path('are', 'P'), f'imgPlot_fig12')
    Plots().periodicFlow_fig(1, saveplot_Path, (ARE_predData, PDS_predData, SD_predData, plotParams), figsize=(11, 6))


args.data_type = 'transient' 
args.exp = 'mthd__snsr'

transient_mthd__snsr()
imgPlot_fig12()


### method and SNR

In [ ]:
def transient_mthd__snr():

  num_epochs, lr, bs, bsv, bn, early_stop  = 1500, 0.0007, 399, 133, 25, 250
  AutoEcoder(bn, lr, num_epochs, bs, bsv, early_stop, drop=0.35)


  for s in [4]:  #[1, 2]:

      num_epochs, lr, hidden_size, seq_len, bn  = 2600, 0.0008, 50, 7, 25
      ARE_train_LSTM(s, seq_len, bn, hidden_size, num_epochs, lr, drop=0.35)

      num_epochs, lr, bn  = 1500, 0.0009, 25
      PDS_train(s, bn, num_epochs, lr)

      num_epochs, lr, wd  = 1700, 0.0009, 1e-4
      SD_train(s, num_epochs, lr, wd, drop=0.1)


  SNR_range = [28]  #range(4, 84, 4)
  error = np.zeros((3, len(SNR_range)))
  plot_s_list = np.array([1, 2, 4])
  plot_seq_len_list = np.array([7])
  plot_image_idx = np.array([335])
  plot_SNR = np.array([28, 76])
  
  for s in [2]: # [1, 2]
    for SNR, i in zip(SNR_range, count(0, 1)):
      seq_len, bn, plot, save = 7, 25, (1, 1, 1), (1, 1, 1)
      print('no. of sensors:', s, ' SNR:', SNR)

      er1 = ARE_test(s, seq_len, bn,
                    plot, save,
                    drop=0.35, SNR=SNR,
                    plot_s_list=plot_s_list, 
                    plot_seq_len_list=plot_seq_len_list,
                    plot_image_idx=plot_image_idx,
                    plot_SNR=plot_SNR)
      
      er2 = PDS_test(s, bn, plot, save,
                    SNR=SNR,
                    plot_s_list=plot_s_list,
                    plot_image_idx=plot_image_idx,
                    plot_SNR=plot_SNR)

      er3 = SD_test(s, plot, save,
                    drop=0.1, SNR=SNR,
                    plot_s_list=plot_s_list,
                    plot_image_idx=plot_image_idx,
                    plot_SNR=plot_SNR)
      
      

      error[:, i] = [er1, er2, er3]

    error_path = get_path('', 'P')
    np.savetxt(osp.join(error_path, "error_{}_{}.csv".format(args.RNN, s)), error, delimiter=",")


def imgPlot_fig13():

    n_sensor = 2
    seq_len = 7
    SNR = 28
    try:
        predData_Path = osp.join(get_path('are', 'W'), f'predData_s{n_sensor}_seqlen{seq_len}_SNR{SNR}.hdf5')
        ARE_predData = h5py.File(predData_Path, 'r')
        print(f'loaded ARE pred data')

        predData_Path = osp.join(get_path('pds', 'W'), f'predData_s{n_sensor}_SNR{SNR}.hdf5')
        PDS_predData = h5py.File(predData_Path, 'r')
        print(f'loaded PDS pred data')

        predData_Path = osp.join(get_path('sd', 'W'), f'predData_s{n_sensor}_SNR{SNR}.hdf5')
        SD_predData = h5py.File(predData_Path, 'r')
        print(f'loaded SD pred data')
    except:
        raise Exception(FileNotFoundError)

    plotParams = 325  # index of image to be plotted as data contains many predictions
    saveplot_Path = osp.join(get_path('are', 'P'), f'imgPlot_fig13')
    Plots().periodicFlow_fig(1, saveplot_Path, (ARE_predData, PDS_predData, SD_predData, plotParams), figsize=(11, 6))

args.data_type = 'transient' 
args.exp = 'mthd__snr'

transient_mthd__snr()
imgPlot_fig13()

### sensor and sequence len

In [ ]:
def transient_snsr__seq_len():
  args.data_type = 'transient' 
  args.exp = 'snsr__seq_len'

  num_epochs, lr, bs, bsv, bn, early_stop = 1300, 0.0007, 399, 133, 25, 200
  AutoEcoder(bn, lr, num_epochs, bs, bsv, early_stop, drop=0)

  for s in [1, 2, 5, 10]:
    for seq_len in [3, 5, 7, 9]:

        num_epochs, lr, hidden_size, s, bn  = 2700, 0.0008, 50, s, 25
        ARE_train_LSTM(s, seq_len, bn, hidden_size, num_epochs, lr, drop=0.35)

  error = np.zeros((4, 4))
  plot_s_list = np.array([1, 2])
  plot_seq_len_list = np.array([7])
  plot_image_idx = np.array([335])  
  plot_SNR = np.array([28, 76])

  for SNR in [None]:  
    for s, j in zip([2], count(0, 1)):  # [1, 2, 5, 10]
      for seq_len, i in zip([5], count(0, 1)):  # [3, 5, 7, 9]
        bn, plot, save = (25, (1, 1, 0), (0, 0, 0))
        print('\n no. of sensors:',s,' seq_len:',seq_len)
        
        er1 = ARE_test(s, seq_len, bn,
                       plot, save,
                       plot_s_list=plot_s_list, 
                       plot_seq_len_list=plot_seq_len_list,
                       plot_image_idx=plot_image_idx)
        
        # er1 = ARE_test(s, seq_len, bn,
        #                plot, save,
        #                drop=0.35, SNR=SNR,
        #                plot_s_list=plot_s_list, 
        #                plot_seq_len_list=plot_seq_len_list,
        #                plot_image_idx=plot_image_idx,
        #                plot_SNR=plot_SNR)

        error[j, i] = er1

    # error_path = get_path('', 'P')
    # np.savetxt(osp.join(error_path, "error_{}_SNR{}.csv".format(args.RNN, SNR)), error, delimiter=",")

transient_snsr__seq_len()

## SST

### method and sensor at certain noise level

In [ ]:
def sst_mthd__snsr_with_noise():

    num_epochs, lr, bs, bsv, bn, early_stop  = 1300, 0.0007, 400, 100, 25, 100
    AutoEcoder(bn, lr, num_epochs, bs, bsv, early_stop, drop=0.35)

    for s in [4, 8, 16, 32]:

        num_epochs, lr, hidden_size, seq_len, bn  = 700, 0.0006, 50, 7, 25
        ARE_train_LSTM(s, seq_len, bn, hidden_size, num_epochs, lr, drop=0.35)

        num_epochs, lr, bn  = 700, 0.0007, 25
        PDS_train(s, bn, num_epochs, lr)

        num_epochs, lr, wd  = 600, 0.0009, 1e-4
        SD_train(s, num_epochs, lr, wd, drop=0.1)

    error = np.zeros((3, 4))
    plot_s_list = np.array([4, 8])
    plot_seq_len_list = np.array([7])
    plot_image_idx = np.array([20])  
    plot_SNR = np.array([20])

    for SNR in [20]: #[10, 20, 30]:  
        for s, i in zip([4], count(0, 1)):  #zip([4, 8, 16, 32], count(0, 1)):  
            seq_len, bn, plot, save, use_stats = 7, 25, (1, 1, 1), (1, 1, 1), 1

            er1 = ARE_test(s, seq_len, bn,
                            plot, save,
                            drop=0.35, SNR=SNR,
                            plot_s_list=plot_s_list, 
                            plot_seq_len_list=plot_seq_len_list,
                            plot_image_idx=plot_image_idx,
                            plot_SNR=plot_SNR,
                            use_stats=use_stats)
            
            er2 = PDS_test(s, bn, plot, save,
                            SNR=SNR,
                            plot_s_list=plot_s_list,
                            plot_image_idx=plot_image_idx,
                            plot_SNR=plot_SNR,
                            use_stats=use_stats)

            er3 = SD_test(s, plot, save,
                            drop=0.1, SNR=SNR,
                            plot_s_list=plot_s_list,
                            plot_image_idx=plot_image_idx,
                            plot_SNR=plot_SNR,
                            use_stats=use_stats)

            error[:, i] = [er1, er2, er3]

        error_path = get_path('', 'P')
        np.savetxt(osp.join(error_path, "error_{}_seq_len{}_SNR{}.csv".format(args.RNN, seq_len, SNR)), error, delimiter=",")


def imgPlot_fig17():

    try:
        predData_Path = osp.join(get_path('are', 'W'), f'predData.hdf5')
        ARE_predData = h5py.File(predData_Path, 'r')
        print(f'loaded ARE pred data')

        predData_Path = osp.join(get_path('pds', 'W'), f'predData.hdf5')
        PDS_predData = h5py.File(predData_Path, 'r')
        print(f'loaded PDS pred data')

        predData_Path = osp.join(get_path('sd', 'W'), f'predData.hdf5')
        SD_predData = h5py.File(predData_Path, 'r')
        print(f'loaded SD pred data')
    except:
        raise Exception(FileNotFoundError)

    plotParams = 20
    saveplot_Path = osp.join(get_path('are', 'P'), f'imgPlot_fig17')
    Plots().sst_fig(1, saveplot_Path, (ARE_predData, PDS_predData, SD_predData, plotParams))

args.data_type = 'sea_temp' 
args.exp = 'mthd__snsr_with_noise'

sst_mthd__snsr_with_noise()
imgPlot_fig17()

### method and bottle neck

In [ ]:
def sst_mthd__bn():
  args.data_type = 'sea_temp' 
  args.exp = 'mthd__bn'

  for bn in [5, 15, 25, 50]:

    num_epochs, lr, bs, bsv, bn, early_stop  = 1300, 0.0007, 400, 100, bn, 100
    AutoEcoder(bn, lr, num_epochs, bs, bsv, early_stop, drop=0.35)

    for s in [2, 4, 8]:

      num_epochs, lr, hidden_size, seq_len, bn  = 700, 0.0006, 50, 7, bn
      ARE_train_LSTM(s, seq_len, bn, hidden_size, num_epochs, lr, drop=0.35)

      num_epochs, lr, bn  = 700, 0.0009, bn
      PDS_train(s, bn, num_epochs, lr)

  error = np.zeros((2, 4))
  plot_s_list = np.array([2])
  plot_seq_len_list = np.array([7])
  plot_image_idx = np.array([40])  # 5
  plot_SNR = np.array([20])
  
  for s in [2, 4, 8]:
    for bn, i in zip([5, 15, 25, 50], count(0, 1)): 

      seq_len, bn, plot, save, use_stats, SNR = 7, bn, (1, 1, 1), (1, 1, 1), 1, 20

      er1 = ARE_test(s, seq_len, bn,
                    plot, save,
                    drop=0.35, SNR=SNR,
                    plot_s_list=plot_s_list, 
                    plot_seq_len_list=plot_seq_len_list,
                    plot_image_idx=plot_image_idx,
                    plot_SNR=plot_SNR)
      
      er2 = PDS_test(s, bn, plot, save,
                    SNR=SNR,
                    plot_s_list=plot_s_list,
                    plot_image_idx=plot_image_idx,
                    plot_SNR=plot_SNR)
      
      error[:, i] = [er1, er2]

    error_path = get_path('', 'P')
    np.savetxt(osp.join(error_path, "error_{}_{}.csv".format(args.RNN, s)), error, delimiter=",")


sst_mthd__bn()

### method and SNR

In [ ]:
def sst_mthd__snr():
  args.data_type = 'sea_temp' 
  args.exp = 'mthd__snr'

  num_epochs, lr, bs, bsv, bn, early_stop  = 1300, 0.0007, 400, 100, 25, 100
  AutoEcoder(bn, lr, num_epochs, bs, bsv, early_stop, drop=0.35)


  for s in [2, 4, 8]:  

      num_epochs, lr, hidden_size, seq_len, bn  = 700, 0.0006, 50, 7, 25
      ARE_train_LSTM(s, seq_len, bn, hidden_size, num_epochs, lr, drop=0.35)

      num_epochs, lr, bn  = 700, 0.0009, 25
      PDS_train(s, bn, num_epochs, lr)

      num_epochs, lr, wd  = 500, 0.0009, 1e-4
      SD_train(s, num_epochs, lr, wd, drop=0.1)

  SNR_range = range(5, 80, 5)
  error = np.zeros((3, len(SNR_range)))
  plot_s_list = np.array([4])
  plot_seq_len_list = np.array([7])
  plot_image_idx = np.array([20])
  plot_SNR = np.array([10, 70])
  
  for s in [2, 4, 8]:
    for SNR, i in zip(SNR_range, count(0, 1)):
      seq_len, bn, plot, save, use_stats = 7, 25, (0, 0, 0), (0, 0, 0), 1

      er1 = ARE_test(s, seq_len, bn,
                    plot, save,
                    drop=0.35, SNR=SNR,
                    plot_s_list=plot_s_list, 
                    plot_seq_len_list=plot_seq_len_list,
                    plot_image_idx=plot_image_idx,
                    plot_SNR=plot_SNR)
      
      er2 = PDS_test(s, bn, plot, save,
                    SNR=SNR,
                    plot_s_list=plot_s_list,
                    plot_image_idx=plot_image_idx,
                    plot_SNR=plot_SNR)

      er3 = SD_test(s, plot, save,
                    drop=0.1, SNR=SNR,
                    plot_s_list=plot_s_list,
                    plot_image_idx=plot_image_idx,
                    plot_SNR=plot_SNR)

      error[:, i] = [er1, er2, er3]

    error_path = get_path('', 'P')
    np.savetxt(osp.join(error_path, "error_{}_{}.csv".format(args.RNN, s)), error, delimiter=",")

sst_mthd__snr()

## Burgers1D

### method and sensors

In [ ]:
def burgers1D_mthd__snsr():

    num_epochs, lr, bs, bsv, bn, early_stop  = 500, 0.0009, 51*3, 1, 16, 200
    AutoEcoder(bn, lr, num_epochs, bs, bsv, early_stop, drop=0)


    for s in [10]: # [10]

        num_epochs, lr, hidden_size, seq_len, bn, bs  = 800, 0.001, 50, 7, 16, 1
        ARE_train_LSTM(s, seq_len, bn, hidden_size, num_epochs, lr, drop=0)

        num_epochs, lr, bn  = 800, 0.001, 25
        PDS_train(s, bn, num_epochs, lr)

        num_epochs, lr, wd  = 800, 0.001, 1e-4
        SD_train(s, num_epochs, lr, wd)

    
    # --------------------------------------------------------------------------
    plot_s_list = np.array([3, 2, 4, 5, 8, 10])
    plot_seq_len_list = np.array([7, 9])
    plot_image_idx = np.array([15, 23, 42])  
    sensor_ls = [10]#[8, 10, 12]

    err_ls = []
    for i, s in enumerate(sensor_ls):
        seq_len, bn, plot, save = 7, 16, (1, 1, 1), (1, 1, 1)

        err = ARE_test(s, seq_len, bn,
                    plot, save,
                    hidden_size=50,
                    plot_s_list=plot_s_list, 
                    plot_seq_len_list=plot_seq_len_list,
                    plot_image_idx=plot_image_idx)
        err_ls.append(err)

    graphData = {}
    graphData[token_hex(2)] =  {'legend': f'ARE',
                                'data': err_ls,
                                'color': 'red',
                                'x_ticks_lables': sensor_ls}
    append_graph(graphData)

    # --------------------------------------------------------------------------
    err_ls = []
    for i, s in enumerate(sensor_ls):
        bn = 25
        err = PDS_test(s, bn, plot, save,
                    plot_s_list=plot_s_list,
                    plot_image_idx=plot_image_idx)
        err_ls.append(err)

    graphData = {}
    graphData[token_hex(2)] =  {'legend': f'PDS',
                                'data': err_ls,
                                'color': 'blue',
                                'x_ticks_lables': sensor_ls}
    append_graph(graphData)

    # --------------------------------------------------------------------------
    err_ls = []
    for i, s in enumerate(sensor_ls):
        err = SD_test(s, plot, save,
                    plot_s_list=plot_s_list,
                    plot_image_idx=plot_image_idx)
        err_ls.append(err)

    graphData = {}
    graphData[token_hex(2)] =  {'legend': f'SD',
                                'data': err_ls,
                                'color': 'green',
                                'x_ticks_lables': sensor_ls}
    append_graph(graphData)


def imgPlot_fig100():
    n_sensor = 10
    seq_len = 7
    SNR = None
    try:
        predData_Path = osp.join(get_path('are', 'W'), f'predData_s{n_sensor}_seqlen{seq_len}_SNR{SNR}.hdf5')
        ARE_predData = h5py.File(predData_Path, 'r')
        print(f'loaded ARE pred data')

        predData_Path = osp.join(get_path('pds', 'W'), f'predData_s{n_sensor}_SNR{SNR}.hdf5')
        PDS_predData = h5py.File(predData_Path, 'r')
        print(f'loaded PDS pred data')

        predData_Path = osp.join(get_path('sd', 'W'), f'predData_s{n_sensor}_SNR{SNR}.hdf5')
        SD_predData = h5py.File(predData_Path, 'r')
        print(f'loaded SD pred data')
    except:
        raise Exception(FileNotFoundError)

    
    saveplot_Path = osp.join(get_path('are', 'P'), f'imgPlot_fig102')

    # plotParams = 39
    # Plots().burgers1D_imgPlot(1, saveplot_Path, (ARE_predData, PDS_predData, SD_predData, plotParams), figsize=(14, 6))
    # pdb.set_trace()
    # plotParams = [23, 29, 39, 47] 161
    plotParams = [23, 38, 113, 183] 
    ARE_predData['pred'].shape[0]
    Plots().burgers1D_imgPlot1(1, saveplot_Path, (ARE_predData, PDS_predData, SD_predData, plotParams), figsize=(14, 6))


def graphPlot_fig_burgers1D_mthd():
    error_path = osp.join(get_path('', 'P'), 'error.yaml')
    errorPlots = yaml.load(open(error_path, 'r'))
    plotParams = {'xlabel': 'No. of sensors', 'ylabel': 'Prediction error'}
    Plots().transientFlow_graphPlot_senLoc(1, osp.join(get_path('', 'P'), 'error'), errorPlots, plotParams)
    


args.data_type = 'burgers1D' 
args.exp = 'mthd__snsr'

burgers1D_mthd__snsr()
imgPlot_fig100()
graphPlot_fig_burgers1D_mthd()


# Recycle Bin